# 텍스트 데이터와 임베딩

### 과정 소개
자연어 처리(NLP)에서 가장 중요한 출발점은 **텍스트 데이터를 기계가 이해할 수 있는 수치형 데이터로 바꾸는 과정**입니다.  
사람에게는 "언어"가 자연스럽지만, 기계에게는 단순한 문자열일 뿐입니다.  
따라서 언어의 의미를 잃지 않으면서 벡터로 바꾸는 기술, 즉 **임베딩(Embedding)** 은 NLP의 핵심입니다. 

## 1. 자연어 처리 소개

### 1.1 자연어 처리란?
- 컴퓨터가 인간의 언어를 이해·생성·활용할 수 있도록 하는 기술  
- 예: 번역기, 챗봇, 음성 비서, 검색 엔진  

### 1.2 왜 중요한가?
- 오늘날 대부분의 데이터는 **비정형 데이터** (텍스트, 이미지, 음성)  
- 텍스트 데이터는 고객 리뷰, 뉴스, 논문 등 **의사결정에 직결되는 정보**를 담고 있음  
- 기계학습/딥러닝이 성공적으로 작동하려면 이 텍스트를 숫자로 잘 표현해야 함  

### 1.3 변천사
- 규칙 기반 → 통계적 모델 → 신경망 기반 → 임베딩+RNN → Transformer  
- 이번 강의는 **임베딩과 RNN까지** 다루며, 다음 장에서 Transformer로 확장 예정  



## 2. 텍스트 데이터 다루기

### 2.1 데이터 정제 (Cleaning)
텍스트 데이터에는 사람이 쓰면서 생긴 **불필요한 잡음(noise)** 이 많이 섞여 있습니다.  
이런 잡음을 제거하지 않으면 모델이 쓸데없는 패턴까지 학습해서 성능이 떨어질 수 있습니다.  

#### 정제 대상
- 불필요한 특수문자, 구두점  
- HTML 태그  
- 중복된 공백, 줄바꿈  
- 대소문자 혼재  
- 불용어(stopwords)

####  다양한 예시

1) **특수문자 제거**
```
원문: "안녕??? 오늘 날씨 진짜 좋다~~~^^"
정제: "안녕 오늘 날씨 진짜 좋다"
```

2) **HTML 태그 제거**
```
원문: "<div>이 영화 <b>정말</b> 최고!!!</div>"
정제: "이 영화 정말 최고"
```

3) **중복 공백/개행 제거**
```
원문: "오늘은   점심에    김밥을   먹었다. \n\n 내일도 김밥?"
정제: "오늘은 점심에 김밥을 먹었다. 내일도 김밥?"
```

4) **대소문자 통일**
```
원문: "Apple is Better than apple."
정제(소문자화): "apple is better than apple."
```

5) **불용어 제거** *(전통 ML·IR에서 선택적 / **LLM 파이프라인에선 보통 사용하지 않음**)*  
- **불용어(Stopwords)**: 문장에서 자주 등장하지만 분류·검색 성능에 **한정적으로만** 기여하는 단어 집합.  
  예: 국문 — “나는/그리고/하지만/오늘/에서 …”, 영문 — “the/and/of/to …”  
- 전통 BoW/TF-IDF, IR 인덱싱에서 **차원 축소·노이즈 감소** 목적으로 **선택적** 사용.  
- LLM 파이프라인(사전학습/미세조정/추론)에서는 **보존**하는 것이 일반적.

```
원문: "나는 오늘 점심으로 김밥을 먹었다."
정제(불용어 제거 예 — 전통 ML/IR용): "오늘 점심 김밥 먹었다"
```


👉 이렇게 정제를 통해 **텍스트를 더 단순하고 의미 중심적으로 바꿔야** 이후 단계(토큰화, 임베딩 등)가 효과적으로 작동합니다.

#### 전처리 여부에 따른 워드클라우드 비교

LLM·딥러닝 시대에는 모든 작업에 강도 높은 전처리가 반드시 필요한 것은 아닙니다.  
하지만 **전통적인 워드클라우드(특히 유행이 다소 지난 단순 빈도 기반 시각화)**  에서는  
전처리를 하지 않을 경우, 아래처럼 **노이즈 단어가 매우 크게 등장하는 문제가 생깁니다.**

| ❌ 전처리 없이 만든 워드클라우드(나쁜 예시) | ✔ 전처리를 적용한 워드클라우드(좋은 예시) |
|--------------------------------------------|--------------------------------------------|
| 불용어(예: “그리고”, “오늘”, “나는”)가 과도하게 크게 표시되고, 중복 문자·기호가 그대로 반영되어 핵심 의미가 묻힘 | 불필요 단어·구두점 제거 후 핵심 키워드만 남아 의미 해석이 쉬움 |
| ![https://velog.io/@bluespot9/%EB%82%B4-%EC%B9%B4%ED%86%A1%EB%B0%A9-%EC%9B%8C%EB%93%9C-%ED%81%B4%EB%9D%BC%EC%9A%B0%EB%93%9C-%EB%A7%8C%EB%93%A4%EA%B8%B0](image/안좋은예시.png) | ![https://datascientest.com/en/wordcloud-whats-it-all-about](image/좋은예시.png) |

#### 목적에 따라 전처리 전략은 달라집니다

전처리가 항상 좋은 것은 아닙니다. **무엇을 위해 사용하는가**에 따라 달라집니다.

| 목적 | 전처리 필요성 | 이유 |
|------|----------------|------|
| **전통 워드클라우드, TF-IDF 분석** | 높음 | 단순 빈도 기반이므로 노이즈 단어가 시각화를 망침 |
| **전통 ML(감성분석, 분류 등)** | 중간 | 불용어 제거·토큰 정규화가 도움되기도 함 |
| **LLM 파이프라인(사전학습·미세조정·RAG·챗봇)** | 낮음 | LLM은 문맥 기반 모델이므로 불용어·기호도 의미 구조 해석에 필요 |
| **언어학적 분석(구문·어휘 다양성)** | 매우 낮음 | 원문 보존이 핵심 |

즉, **좋은 전처리란, 모든 것을 없애는 것이 아니라 목적에 맞게 적절히 다듬는 것**입니다.

#### 문제 1. 대소문자 통일
문장 `"Machine Learning is FUN and Useful."`을 모두 소문자로 바꿔보세요.  

<details>
<summary>정답 보기</summary>

```python
text = "Machine Learning is FUN and Useful."
print(text.lower())
# 출력: "machine learning is fun and useful."
```
</details>

In [ ]:
# 여기에 작성하세요
text = "Machine Learning is FUN and Useful."

#### 문제 2. 불용어 제거
문장 `"나는 오늘 아침에 학교에 갔다."`에서 불용어 ["나는", "오늘", "에"]를 제거해보세요.  

<details>
<summary>정답 보기</summary>

```python
text = "나는 오늘 아침에 학교에 갔다."
stopwords = ["나는", "오늘", "에"]

for word in stopwords:
    text = text.replace(word, '')
    
text.strip()
print(text)
# 출력: "아침에 학교에 갔다."
```
</details>

In [ ]:
# 여기에 작성하세요
text = "나는 오늘 아침에 학교에 갔다."
stopwords = ["나는", "오늘", "에"]

### 2.2 정규표현식(Regex)
정규표현식(Regular Expression, **regex**)은 **문자열에서 특정 패턴을 찾고/바꾸고/분리**하는 강력한 도구입니다.  
전자영수증에서 숫자만 뽑아내고, 로그에서 IP만 추출하고, 텍스트 노이즈를 빠르게 정리하는 등 **대량의 텍스트 처리 자동화**에 핵심적으로 쓰입니다.


#### 왜 Regex를 쓰나요?
- **일관된 패턴**을 한 번에 처리 (이메일, URL, 날짜, 숫자, 해시태그 등)
- **간결한 코드**로 복잡한 문자열 조작 수행
- 전처리(클리닝) 단계에서 **재사용 가능한 규칙**으로 품질 유지


#### 핵심 문법(요약)
- **문자클래스**: `\d`(숫자), `\D`(숫자 아님), `\w`(단어문자: [A-Za-z0-9_]), `\s`(공백)  
- **반복/수량자**: `*`(0+), `+`(1+), `?`(0 or 1), `{m,n}`(m~n회)  
- **그룹/선택**: `( )`(캡처 그룹), `(?: )`(비캡처), `|`(OR)  
- **앵커**: `^`(문자열/행 시작), `$`(문자열/행 끝), `\b`(단어 경계)  
- **플래그**: `re.I`(대소문자 무시), `re.M`(멀티라인: ^,$를 행 단위로), `re.S`(dot이 개행 포함)  
- **탐욕/게으름**: `.*`(탐욕적), `.*?`(게으름; 가능한 한 짧게)

> **주의**: 파이썬에서는 `r"..."` **원시 문자열**을 사용해 백슬래시 이스케이프를 피하세요.

#### 정규표현식 주요 패턴 표

| 패턴 | 의미 | 예시 | 매칭 결과 |
|------|------|------|-----------|
| `.` | 임의의 한 문자(개행 제외) | `a.c` | `abc`, `axc` |
| `^` | 문자열/행의 시작 | `^Hi` | `"Hi there"` |
| `$` | 문자열/행의 끝 | `end$` | `"the end"` |
| `\d` | 숫자 (0–9) | `\d{3}` | `123`, `007` |
| `\D` | 숫자가 아닌 문자 | `\D+` | `"abc"`, `"--"` |
| `\w` | 단어문자 `[A-Za-z0-9_]` | `\w+` | `"hello"`, `"Python3"` |
| `\W` | 단어문자가 아닌 것 | `\W+` | `"!!"`, `" "` |
| `\s` | 공백 (스페이스, 탭, 개행) | `a\sb` | `"a b"` |
| `\S` | 공백이 아닌 문자 | `\S+` | `"text"`, `"123"` |
| `*` | 0회 이상 반복 | `ab*` | `"a"`, `"ab"`, `"abbb"` |
| `+` | 1회 이상 반복 | `ab+` | `"ab"`, `"abbb"` |
| `?` | 0회 또는 1회 | `ab?` | `"a"`, `"ab"` |
| `{m,n}` | m~n회 반복 | `\d{2,4}` | `99`, `2025` |
| `( )` | 그룹화 / 캡처 | `(ab)+` | `"ab"`, `"abab"` |
| `(?: )` | 비캡처 그룹 | `(?:ab)+` | `"abab"` |
| `|` | OR 선택 | `cat|dog` | `"cat"`, `"dog"` |
| `\b` | 단어 경계 | `\bcat\b` | `"cat"` (단어 단독일 때) |
| `(?i)` | 대소문자 무시 플래그 | `(?i)abc` | `"abc"`, `"ABC"` |

#### 파이썬 정규표현식 함수 요약

| 함수 | 설명 | 예시 코드 | 결과 |
|------|------|-----------|------|
| `re.match(pattern, string)` | 문자열 **처음부터** 패턴 매칭 | `re.match(r"\d+", "123abc")` | `<Match '123'>` |
| `re.search(pattern, string)` | 문자열 전체에서 **처음 매칭되는 패턴** 찾기 | `re.search(r"\d+", "abc123xyz")` | `<Match '123'>` |
| `re.findall(pattern, string)` | **모든 매칭 결과**를 리스트로 반환 | `re.findall(r"\d+", "a12 b34 c56")` | `['12', '34', '56']` |
| `re.finditer(pattern, string)` | 모든 매칭 결과를 **이터레이터(객체)** 로 반환 | `[m.group() for m in re.finditer(r"\d+", "a12 b34")]` | `['12', '34']` |
| `re.sub(pattern, repl, string)` | 패턴을 다른 문자열로 **치환** | `re.sub(r"\d+", "#", "ID123")` | `"ID#"` |
| `re.split(pattern, string)` | 패턴 기준으로 문자열 **분리** | `re.split(r"\s+", "a b   c")` | `['a', 'b', 'c']` |
| `re.fullmatch(pattern, string)` | 문자열 전체가 패턴과 **완전히 일치**할 때 매칭 | `re.fullmatch(r"\d{3}", "123")` | `<Match '123'>` |
| `re.compile(pattern)` | 정규표현식을 객체로 컴파일 (재사용 최적화) | `p = re.compile(r"\d+")`<br>`p.findall("1a2b3")` | `['1','2','3']` |

> ⚠️ `match`는 문자열의 시작 부분만 확인, `search`는 전체 탐색을 수행한다는 점이 중요합니다.

#### 자주 쓰는 패턴 예시

In [ ]:
### 1) 숫자/기호 제거 + 공백 정규화

import re
text = "오늘은 2025년 9월 10일!!! 날씨   정말 좋다   ^^"
no_digits = re.sub(r"\d+", "", text)               # 숫자 제거
no_punct  = re.sub(r"[^\w\s가-힣]", " ", no_digits) # 기호 제거(한글/영문/숫자/공백만 남김)
cleaned   = re.sub(r"\s+", " ", no_punct).strip()   # 다중 공백 → 단일 공백
print(cleaned)  # "오늘은 년 월 일 날씨 정말 좋다"

In [ ]:
### 2) 이메일 추출

import re
text = "문의: admin@example.com, 혹은 support@my-site.co.kr 로 연락주세요."
emails = re.findall(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[A-Za-z]{2,}", text)
print(emails)  # ['admin@example.com', 'support@my-site.co.kr']

In [ ]:
### 3) URL 제거 (http/https)

import re
text = "공식 문서: https://docs.python.org 참고, 우리 블로그 http://example.com/blog 도 봐요."
no_url = re.sub(r"https?://\S+", "", text).strip()
print(no_url)  # "공식 문서:  참고, 우리 블로그  도 봐요."

In [ ]:
### 4) 한국 전화번호 마스킹

import re
text = "연락처: 010-1234-5678 / 02-345-6789"
masked = re.sub(r"\b(\d{2,3})-(\d{3,4})-(\d{4})\b", r"\1-****-****", text)
print(masked)  # "연락처: 010-****-**** / 02-****-****"

In [ ]:
### 5) 멀티라인에서 행 시작/끝 활용 (`re.M`)

import re
log = "OK: step1\nERROR: step2 failed\nOK: step3"
errors = re.findall(r"^ERROR:.*$", log, flags=re.M)
print(errors)  # ['ERROR: step2 failed']

In [ ]:
### 6) 탐욕 vs 게으름 (HTML 태그 사이 내용 캡처 예시)

import re
html = "<p>첫째</p><p>둘째</p>"
greedy = re.findall(r"<p>.*</p>", html)      # 탐욕적: 한 방에 다 먹음
lazy   = re.findall(r"<p>.*?</p>", html)     # 게으름: 가능한 짧게 두 개로 나눔
print(greedy)  # ['<p>첫째</p><p>둘째</p>']
print(lazy)    # ['<p>첫째</p>', '<p>둘째</p>']

> **HTML 파싱은 정규표현식만으로 완벽히 처리하기 어렵습니다.**  
> 실제 서비스에선 `BeautifulSoup` 같은 파서를 함께 쓰는 걸 권장합니다.

#### 문제 3. 숫자와 기호 제거 + 공백 정규화
문자열에서 **숫자/특수기호를 제거**하고, **다중 공백을 하나로** 바꾼 문자열을 출력하세요.  
문자/숫자/공백/한글만 남기도록 하세요.

<details> <summary>정답 보기</summary>

```python 
import re
text = "정가: 19,800원!!! ★★ 대박 할인 30% ★★  (한정 수량)"
tmp = re.sub(r"\d+", "", text)                         # 숫자 제거
tmp = re.sub(r"[^\w\s가-힣]", " ", tmp)                # 기호 제거
result = re.sub(r"\s+", " ", tmp).strip()             # 공백 정규화
print(result)  # "정가 원 대박 할인 한정 수량"
```
</details>



In [ ]:
# 여기에 작성하세요
import re
text = "정가: 19,800원!!! ★★ 대박 할인 30% ★★  (한정 수량)"

#### 문제 4. 이메일만 추출하기
문장에서 모든 이메일을 찾아 **리스트**로 반환하세요.

<details> <summary>정답 보기</summary>

```python 
import re
text = "메일: kim.ai@univ.ac.kr; 홍보: sales-team@example.com; 오류: bug+test@my.io"
emails = re.findall(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[A-Za-z]{2,}", text)
print(emails)
# ['kim.ai@univ.ac.kr', 'sales-team@example.com', 'bug+test@my.io']
```
</details>



In [ ]:
# 여기에 작성하세요
text = "메일: kim.ai@univ.ac.kr; 홍보: sales-team@example.com; 오류: bug+test@my.io"

#### 문제 5. URL 제거하기
문장에서 **http/https URL을 모두 제거**하세요. (도메인 뒤 공백 정리 포함)

<details> <summary>정답 보기</summary>

```python 
import re
text = "문서: https://a.b/c?x=1  블로그: http://blog.com/post  끝."
no_url = re.sub(r"https?://\S+", "", text)     # URL 제거
no_url = re.sub(r"\s+", " ", no_url).strip()   # 공백 정리
print(no_url)  # "문서: 블로그: 끝."
```
</details>

In [ ]:
# 여기에 작성하세요
text = "문서: https://a.b/c?x=1  블로그: http://blog.com/post  끝."

#### 문제 6. 전화번호 마스킹
문장에서 한국식 전화번호(예: `010-1234-5678`, `02-345-6789`)의 **가운데/끝 4자리**를 `*`로 마스킹하세요.

<details> <summary>정답 보기</summary>

```python
import re
text = "문의: 010-1234-5678 / 대리점: 031-987-6543 / 회사: 02-345-6789"
masked = re.sub(r"\b(\d{2,3})-(\d{3,4})-(\d{4})\b", r"\1-****-****", text)
print(masked)
# "문의: 010-****-**** / 대리점: 031-****-**** / 회사: 02-****-****"
``` 
</details>



In [ ]:
# 여기에 작성하세요
text = "문의: 010-1234-5678 / 대리점: 031-987-6543 / 회사: 02-345-6789"

#### 문제 7. 문장 분리 (간단 버전)
`.` `?` `!` 중 하나로 끝나는 구두점 기준으로 **문장을 분리**하고, 출력 시 앞뒤 공백을 제거하세요.

<details> <summary>정답 보기</summary>

```python 
import re
text = "안녕하세요! NLP 재밌죠? 지금은 정규표현식 중입니다.  예시를 더 볼까요!"
sentences = re.split(r"[.!?]+", text)
sentences = [s.strip() for s in sentences if s.strip()]
print(sentences)
# ['안녕하세요', 'NLP 재밌죠', '지금은 정규표현식 중입니다', '예시를 더 볼까요']
```
</details>

In [ ]:
# 여기에 작성하세요
text = "안녕하세요! NLP 재밌죠? 지금은 정규표현식 중입니다.  예시를 더 볼까요!"

### 2.3 토큰화(Tokenization)

토큰화(Tokenization)는 **텍스트를 작은 단위(토큰, token)로 분리하는 과정**입니다.  
컴퓨터는 텍스트를 숫자로 바꿔야 이해할 수 있는데, 긴 문장을 그대로 하나의 단위로 두면 너무 복잡하고 차원이 폭발합니다.  
따라서 문장을 규칙적으로 잘라 **모델이 처리 가능한 단위**로 나누는 것이 필요합니다.  

<img src="image/tokenization.png" width=700>

#### 왜 토큰화가 중요한가?
- 딥러닝 모델은 보통 **최대 입력 길이 제한**이 있음 → 긴 문장은 적절히 쪼개서 입력해야 함  
- 텍스트를 잘 분리해야 단어·형태·문맥 정보를 더 잘 반영할 수 있음  
- **희귀 단어 문제(OOV, Out Of Vocabulary)** 완화:  
  - 전통 단어 기반 토큰화에서는 사전에 없는 단어(OOV)가 나오면 모델이 처리 불가  
  - Subword 기반 토큰화는 단어를 더 작은 단위로 쪼개어 표현 가능 → **사전에 없는 새로운 단어도 기존 subword들의 조합으로 처리 가능**  

👉 즉, 토큰화는 단순히 “사람이 띄어쓰기 한 단위”가 아니라, **모델이 학습하기 좋은 입력 단위로 나누는 과정**입니다. 

#### 토큰을 너무 잘게 또는 너무 크게 나눌 때 생기는 문제와, Subword가 해답이 되는 이유

토큰화의 핵심은 **각 토큰이 어느 정도 ‘의미 단위’를 갖도록 나누는 것**인데,  
너무 잘게 쪼개버리면 **각 조각이 의미를 잃고 모델이 문맥을 이해하기 훨씬 어려워진다**는 문제가 있습니다.

예를 들어,

- "사랑"이라는 단어는 → **love**라는 뚜렷한 의미가 있음  
- 하지만 너무 잘게 쪼개면  
  - "사" → *사랑, 사람, 사막, 사전, 사고…* 등 무한한 단어의 시작 부분  
  - "랑" → *결코 단독 의미가 없는 조각*  
- 이렇게 되면 **개별 토큰 하나로는 의미를 유추할 수 없고**,  
  모델은 “사 + 랑 = 사랑이라는 의미”라는 사실을 **일일이 학습해서 조합**해야 함.

즉, 너무 잘게 쪼개진 토큰은  
**① 의미가 약하고,  
② 일관성도 없으며,  
③ 모델이 문맥을 스스로 재구성해야 하는 부담이 커진다.**

#### 너무 크게 쪼갤 때의 문제: 사전(Vocabulary) 크기 폭발

이번에는 반대로, 한 단어를 너무 큰 단위로(예: 띄어쓰기 단위 그대로) 토큰화했을 때의 문제를 보겠습니다.  
이 문제를 이해하려면, 먼저 **컴퓨터가 잘라낸 토큰을 어떻게 숫자로 바꾸는지** 그 과정을 알아야 합니다.

**1. 토큰화 후에는 반드시 '출석부'를 만듭니다.**
컴퓨터는 '사과', '배' 같은 글자를 직접 계산할 수 없습니다.   
그래서 잘라낸 토큰들을 모아 중복을 없애고, **고유한 번호(ID)** 를 하나씩 붙여줍니다.

- 우리가 학생들의 이름을 매번 부르는 대신 "1번, 2번" 하고 출석 번호를 부르는 것과 같습니다.
- 이 **"토큰-번호 매칭 리스트"** 를 우리는 **사전(Vocabulary)** 이라고 부릅니다.

**2. 사전(Vocabulary)의 구조**
실제 모델은 텍스트가 들어오면 이 사전을 보고 글자를 숫자로 바꿔서 받아들입니다.

> **[사전 예시]**
> - 0번: `<pad>` (빈 공간)
> - 1번: `<unk>` (모르는 단어)
> - 2번: "사랑"
> - 3번: "사람"
> - ...

**3. 정수 인코딩 (Integer Encoding)**
이제 "나는 사람이다"라는 문장이 들어오면, 컴퓨터는 사전을 참조해 다음과 같이 숫자로 변환합니다.

> "나는 사람이다"
> → 토큰화: `["나는", "사람", "이다"]`
> → 정수 변환: `[10, 3, 52]`  *(각 토큰에 해당하는 사전의 인덱스)*

**여기서 중요한 점은, 사전에 등록된 토큰의 개수(=사전의 크기)가 바로 모델이 다뤄야 할 숫자의 범위가 된다는 것입니다.**

이제, 토큰을 너무 크게(단어 단위로) 잡으면 이 **사전의 크기**에 어떤 일이 생기는지 보면:

- 예를 들어 한국어는 조사·어미 변화가 매우 다양함  
  - "먹다, 먹었다, 먹어라, 먹었습니다, 먹었겠지, 먹고, 먹고는…"  
- 이런 변형된 단어들을 **각각 하나의 토큰(하나의 인덱스)**으로 취급하면:
  - "먹다" → 인덱스 1001  
  - "먹었다" → 인덱스 1002  
  - "먹었습니다" → 인덱스 1003  
  - "먹었겠다" → 인덱스 1004  
  - … 이런 식으로 끝도 없이 늘어남
  
- 결과적으로:
  1) **사전 크기(토큰 종류 수)가 기하급수적으로 커짐**  
     - 사전에 등록해야 할 “인덱스-토큰 쌍”이 너무 많아짐  
  2) 사전이 커지면
     - 모델이 **외워야 할 토큰 종류가 너무 많아져서**  
       → 임베딩(embedding) 행렬 크기가 커지고, 메모리/학습 비용이 증가  
  3) 텍스트에 드물게 등장하는 희귀 형태(예: "먹었겠지요만")는  
     - 사전에 **한 번 밖에 안 나오는 토큰**이 되어  
       → 모델이 그 토큰의 의미를 제대로 학습하기 어려움  
  4) 사전에 등록되지 않은 완전히 새로운 단어(OOV)가 나올 수도 있음  
     - 이 경우 **그 단어 전체를 처리할 수 없음**

정리하면,  
**너무 크게 쪼개서 “단어 변형 하나당 인덱스 하나”를 부여하는 방식은**  

- 사전에 들어가야 할 토큰 종류(=인덱스 개수)가 폭발적으로 증가하고  
- 그만큼 모델의 기억해야 할 것, 학습해야 할 것, 저장해야 할 파라미터가 감당하기 힘들어지는 구조입니다.

#### 그래서 Subword가 좋은 이유: 의미 보존 + 효율성의 균형

Subword 방식은 위 두 가지 극단의 문제를 동시에 완화하는 절충안입니다.

1) **너무 잘게 쪼갤 때의 의미 손실 → 완화**  
   - "사"처럼 의미 없는 최소 단위까지 쪼개지 않음  
   - 대신, 말뭉치(코퍼스)를 분석해서 **자주 등장하고 의미 있는 단위들**을 subword로 선택  
   - 예: "사랑", "사람", "사막"이 자주 나오면,  
     - "사" 하나가 아니라 "사랑", "사람", "사막" 자체 또는 그 안의 유의미한 조각들을 토큰으로 삼음

2) **너무 크게 쪼갤 때의 사전 폭발 → 완화**  
   - “단어 단위 전체”를 무조건 하나의 토큰으로 만들지 않음  
   - 대신, 적당한 subword 단위를 사전에 등록해 두고,  
     새로운 단어가 등장하면 **기존 subword들을 조합해서 표현**  
   - 예: "자연어처리학개론"이 처음 나와도  
     - ["자연", "어", "처리", "학", "개론"]처럼 기존에 있는 조각들로 나눠서 인덱스로 변환 가능

결국 Subword 토큰화는:

- **너무 잘게** → 토큰 하나에 의미가 거의 없음 → 문맥 이해 어려움  
- **너무 크게** → 토큰 하나당 인덱스가 너무 많아짐 → 사전 폭발 & 학습 비효율  

이 두 문제 사이에서, **각 토큰이 어느 정도 의미를 가지면서도, 사전 크기를 관리 가능한 수준으로 유지**하게 해 주는 방법입니다.

그래서 현대 NLP에서는 **Subword 기반 토큰화(BPE, SentencePiece 등)** 가  
사실상 표준처럼 널리 사용되고 있습니다.

#### 직관적 비유
- 긴 텍스트는 한 번에 삼키기 어려운 **큰 빵**과 같다.  
- 토큰화는 빵을 적당히 잘라서 **한 입 크기 조각**으로 만드는 과정.  
- 조각이 너무 크면 먹기 힘들고, 너무 잘게 부수면 원래의 맛을 잃을 수 있음.  
- Subword 방식은 “적당한 크기의 조각”을 만들어, 모델이 새로운 빵(신조어, 희귀 단어)도 부담 없이 먹게 해줌.  


#### 정리
- 토큰화는 NLP 파이프라인의 출발점  
- 목적: 모델이 처리할 수 있는 단위로 텍스트를 나누기 위함  
- 단어 기반 → 단순하지만 한계 존재  
- 형태소 기반 → 한국어 같은 교착어에 유리  
- Subword 기반 → 희귀 단어·신조어 대응

#### 실습: SentencePiece로 Subword 나누기

지금까지는 토큰화와 Subword의 개념을 직관적으로 살펴봤습니다.  
이제 SentencePiece를 사용해 실제로 Subword 사전을 만들어 보겠습니다.

#### 1) 설치
```bash
pip install sentencepiece
```

#### 2) 간단한 학습용 데이터 준비

##### `data/sample.txt`

아래는 학습용 말뭉치 파일(`data/sample.txt`)의 일부 예시입니다.  
자연어 처리, 토큰화, 모델 학습과 관련된 문장들이 다양하게 포함되어 있습니다.

```
자연어 처리는 재미있다.
자연어 처리를 배우는 것은 유익하다.
나는 오늘도 자연어 처리를 공부한다.
...

   (중략)

...
텍스트 요약은 중요한 정보를 짧게 줄이는 기술이다.
자연어 처리는 음성 인식이나 번역에도 응용된다.
딥러닝의 발전은 자연어 처리 성능을 크게 향상시켰다.
나는 앞으로도 NLP 분야를 꾸준히 공부할 예정이다.
```

> 위와 같이 자연어 처리 관련 표현, 조사·어미 변화, 다양한 문장 구조 등이 포함된 학습용 샘플 텍스트입니다.


#### 3) SentencePiece 모델 학습

- `--model_type`은 `bpe`, `unigram`, `char`, `word` 등 선택 가능
- 여기서는 BPE(Byte Pair Encoding) 사용

In [ ]:
import sentencepiece as spm

# SentencePiece 모델 학습
spm.SentencePieceTrainer.train(
    input="data/sample.txt",       # 학습에 사용할 텍스트 파일 경로
    model_prefix="data/spm",       # 출력 파일 이름 prefix (spm.model, spm.vocab 생성)
    vocab_size=220,                # 생성할 토큰(서브워드) 개수. 최소 문자+메타토큰 이상으로 설정해야 함
    model_type="bpe",              # 사용할 토큰화 알고리즘: bpe / unigram / char / word 중 선택
    character_coverage=1.0         # 학습 데이터에 등장하는 문자를 100% 포함하도록 설정 (한국어는 1.0 권장)
)

#### 4) 학습된 모델 불러오기 & 토큰화

In [ ]:
import sentencepiece as spm

sp = spm.SentencePieceProcessor()
sp.load("data/spm.model")

# 테스트 문장
text = "자연어처리학개론은 흥미롭다."

# 토큰화
tokens = sp.encode_as_pieces(text)
ids = sp.encode_as_ids(text)

print("입력 문장:", text)
print("Subword 토큰:", tokens)
print("토큰 ID:", ids)

**출력 예시**
```
입력 문장: 자연어처리학개론은 흥미롭다.
Subword 토큰: ['▁자연어', '처리', '학개론', '은', '▁', '흥', '미', '롭', '다', '.']
토큰 ID: [7, 43, 0, 74, 53, 0, 68, 0, 56, 54]

#### 문제 8. 단어 단위 토큰화
문장 `"오늘은 자연어 처리를 공부한다."`를 **띄어쓰기 기준**으로 토큰화하세요.  

<details> <summary>정답 보기</summary>

```python
sentence = "오늘은 자연어 처리를 공부한다."
tokens = sentence.split()
print(tokens)
# 출력: ['오늘은', '자연어', '처리를', '공부한다.']
```
</details>

In [ ]:
# 여기에 정답을 작성하세요
sentence = "오늘은 자연어 처리를 공부한다."

#### 문제 9. Subword 토큰화의 장점
문장 `"자연어처리학개론"` 같은 긴 단어가 있을 때, Subword 토큰화의 장점을 설명하세요.  

<details> <summary>정답 보기</summary>

- 긴 복합어가 그대로 단어 사전에 있으면 **희귀 단어(OOV) 문제** 발생  
- Subword 토큰화는 긴 단어를 작은 단위로 분리하여 OOV 문제를 줄임  
- 신조어나 처음 보는 단어도 subword 조합으로 표현 가능  
</details>


### 2.4 전통적 텍스트 인코딩(Traditional Text Encoding)

과거 NLP에서는 토큰화 후, 각 단어를 **모델이 계산할 수 있는 벡터**로 변환하기 위해  
다음과 같은 **연속된 인코딩 파이프라인**을 사용했습니다.

#### 1) 단어 사전(Dictionary) 생성 + 정수 ID 부여

먼저 개발자가 직접 단어 사전을 만들고, 각 단어에 고유한 정수 ID를 부여합니다.

예:
```
{"나는": 1, "오늘": 2, "밥을": 3, "맛있게": 4, "먹었다": 5}
```

이 정수 ID는 단어를 **구분하기 위한 식별자(identifier)** 일 뿐입니다.  
하지만 중요한 점은 다음과 같습니다:

- 정수 1, 2, 3, 4, 5는 **단어의 의미적 크기나 순서를 반영하지 않음**
- ID만으로는 **벡터 연산에 바로 사용할 수 없음**
- 모델이 “ID 5 > ID 1”처럼 **잘못된 규칙성을 학습할 위험**이 있음

따라서 정수 ID를 그대로 모델 입력으로 사용할 수 없었고,  
이를 벡터로 확장하는 추가적인 인코딩 과정이 필요했습니다.



#### 2) 원-핫 인코딩(One-hot Encoding)

정수 ID를 벡터로 변환하는 전통적 방식이 바로 **원-핫 인코딩**입니다.

단어 사전 크기(vocab_size)만큼의 벡터를 만들고,  
해당 단어 ID 위치에만 1을 두고 나머지는 0으로 채웁니다.

예: vocab_size=5

| 단어     | 나는 | 오늘 | 밥을 | 맛있게 | 먹었다 |
|----------|------|------|------|--------|--------|
| 나는     | 1    | 0    | 0    | 0      | 0      |
| 오늘     | 0    | 1    | 0    | 0      | 0      |

이 방식의 특징:

- 단어 간 의미 유사성이 **전혀 반영되지 않음**  
- 벡터 길이가 vocab_size여서 **고차원**  
- 대부분 0이므로 **희소(sparse)**  

즉, 원-핫은 단어를 “하나만 1인 표시 등”으로 구분할 뿐,  
단어의 **의미를 담는 표현**이 아닙니다.

#### 3) 원-핫 벡터와 임베딩 레이어(Embedding Layer)

원-핫 벡터는 그 자체로도 모델의 입력으로 사용될 수 있습니다(예: 초기 RNN, 간단한 분류 모델).   
하지만 **희소성(Sparsity)** 과 **차원 폭발** 문제 때문에, 딥러닝에서는 이를 **밀집 벡터(Dense Vector)**   
로 변환하는 **임베딩 레이어**를 주로 사용합니다.

즉, 원-핫 벡터를 **그대로 학습 데이터로 쓰기보다는,  
임베딩 레이어를 통과시키기 위한 '재료(인덱스 스위치)'로 사용**하는 방식입니다.

##### ✔ 원-핫 벡터의 shape
```
(1 × vocab_size) : 단어(토큰) 하나일 때
(seq_len × vocab_size) : 문장 전체일 때 (seq_len: 문장의 토큰 수)
```

##### ✔ 임베딩 레이어의 shape  
임베딩 레이어는 다음과 같은 거대한 행렬입니다:

```
(vocab_size × embedding_dim)
```

예: vocab_size=10, embedding_dim=512  
→ 임베딩 행렬 shape = (10 × 512)

##### ✔ 원-핫 × 임베딩 = 토큰 임베딩 벡터

행렬 곱:
```
(seq_len × vocab_size) × (vocab_size × 512) = (seq_len × 512)
```

즉:

- 원-핫 벡터는 **임베딩 행렬의 특정 행(row)을 선택하는 스위치**
- 결과로 **토큰 하나가 512차원 벡터**로 변환됨

#### 정리: 전통 NLP의 인코딩은  
#### **정수 ID → 원-핫 → 임베딩 레이어”의 연속된 하나의 과정**

전통 방식의 전체 흐름을 요약하면 다음과 같습니다:

```
정수 ID
  ↓ (위치만 표시)
원-핫 벡터
  ↓ (행렬 곱)
임베딩 벡터 (1 × embedding_dim)
```

여기서 임베딩 벡터가 **실제로 모델이 학습하고 사용하는 표현**입니다.

#### 참고: 현대 NLP에서는 이 과정이 완전히 자동화된다

SentencePiece, WordPiece(BERT), GPT 토크나이저는

- 토큰화  
- 정수 ID 변환  
- 특수 토큰 처리  

까지 모두 자동으로 수행하며, 이 ID는 원-핫을 만들지 않고  
곧바로 **임베딩 레이어의 행(row)을 조회(lookup)** 하여 벡터를 얻습니다.

현대 방식:
```
정수 ID → 임베딩 레이어 행 선택 → (1 × embedding_dim)
```
전통 방식의 행렬 곱 연산을  
효율화한 형태라고 이해하면 됩니다.

<img src="image/embedding_visual.png" width="600">

이미지 출처: 김민수 강사가 AI로 생성

### 💡 핵심 요약

- 전통적 방식은  
  **정수 ID → 원핫 → 임베딩 레이어**로 구성된 연속적인 인코딩 파이프라인  
- 원-핫 벡터는 “표현”이 아니라 **임베딩을 선택하는 스위치**  
- 임베딩 레이어 shape은 `(vocab_size × embedding_dim)`  
- 토큰 하나는 최종적으로 `(1 × embedding_dim)` 벡터가 되어 모델에 전달됨  
- 현대 방식은 원-핫을 생략하고 **ID로 바로 lookup**하여 효율적으로 처리

## 3. 임베딩 (Embedding)

임베딩은 텍스트(단어, 문장, 문서 등)를 **의미를 반영하는 숫자 벡터로 표현**하는 방법입니다.  
단순히 “숫자로 바꿨다”가 아니라, 벡터 공간 위에서 **단어들 사이의 유사도와 관계**가 드러나도록 만드는 것이 핵심입니다.

이전 섹션에서 우리는 텍스트를 숫자로 바꾸기 위한 여러 **인코딩(encoding)** 방법들을 봤습니다.  
이제부터는 한 단계 더 나아가, **의미를 잘 담은 벡터 표현 = 임베딩(embedding)** 을 살펴봅니다.

### 3.1 왜 임베딩이 필요한가?

#### 1) 기존 인코딩의 한계 (리마인드)

- **정수 인코딩(Integer Encoding)**  
  - 단어마다 번호(ID)만 붙이는 방식입니다.  
    예) `{"강아지": 1, "고양이": 2, "자동차": 3, ...}`  
  - 숫자의 크기와 단어의 의미는 아무 관계가 없는데,  
    모델 입장에서는 `1 < 2 < 3` 같은 **가짜 순서 정보**를 학습할 위험이 있습니다.

- **원-핫 인코딩(One-hot Encoding)**  
  - 단어 사전 크기가 `V`일 때, 길이가 `V`인 벡터에서 해당 단어 위치만 1이고 나머지는 모두 0인 벡터를 사용합니다.
  - 모든 벡터가 서로 직교(orthogonal)라서 **단어 사이의 유사성이 전혀 표현되지 않습니다.**
  - 벡터가 매우 길고 대부분이 0인 **희소(sparse) 벡터**라서, 계산 효율도 좋지 않습니다.

```
어휘 사전: ["강아지", "고양이", "자동차", "사과", "바나나"]
"고양이" = [0, 1, 0, 0, 0]
"강아지" = [1, 0, 0, 0, 0]
→ 두 벡터의 내적 = 0 (완전히 다른 방향)
→ 실제로는 비슷한 동물인데, 벡터 상에서는 아무 관련이 없음
```
정리하면, 기존 인코딩은 “컴퓨터가 처리할 수 있도록 텍스트를 숫자로 바꾸는 것”에는 성공했지만,  
“단어의 의미나 단어들 사이의 관계를 표현하는 것”에는 한계가 있음

#### 2) 임베딩 벡터의 아이디어
- 임베딩은 이런 한계를 극복하기 위해 등장했습니다.
- **벡터의 차원을 줄이고(dense)** 벡터의 각 차원이 단어의 **의미**, 문맥, 역할 등을 반영하도록 학습시키는 것이 목표입니다.
- 즉, 의미가 비슷한 단어는 벡터 공간에서도 가깝게, 의미가 다른 단어는 멀리 떨어지도록 만드는 것입니다.

```
"강아지" = [ 0.13, -0.25,  0.77,  0.05]
"고양이" = [ 0.12, -0.33,  0.80,  0.04]
"자동차" = [-0.80,  0.10, -0.12,  0.55]
→ "강아지"와 "고양이" 벡터는 서로 가깝고, "자동차" 벡터는 이 둘과 멀리 떨어져 있도록 학습

이렇게 되면, 비슷한 단어끼리는 군집(clustering)이 생기고
단어 간 관계를 벡터 연산으로도 어느 정도 표현할 수 있습니다.
(예: “왕 - 남자 + 여자 ≈ 여왕” 같은 Word2Vec의 예시)

```

#### 3) 인코딩 vs 임베딩 비교

| 구분             | 정수 인코딩        | 원-핫 인코딩          | 임베딩(Embedding)              |
|------------------|--------------------|------------------------|--------------------------------|
| 표현 방식        | 단순 번호(ID)      | 희소 벡터(sparse)      | 밀집 벡터(dense)              |
| 차원 크기        | 1                  | 단어 개수만큼          | 보통 수십~수백 차원           |
| 단어 간 유사성   | 반영 불가          | 반영 불가              | 벡터 거리/각도로 반영 가능    |
| 공간 효율        | 높음               | 낮음 (차원 ↑)          | 비교적 높음 (차원 ↓)          |
| 학습 방식        | 규칙만 정의        | 규칙만 정의            | **데이터로부터 학습**         |

- **인코딩**: “텍스트를 숫자로 바꾸는 최소한의 단계”  
- **임베딩**: “숫자 벡터 안에 **의미와 관계**까지 담아내는 표현”

이 섹션(3장)에서는 임베딩의 개념을 **발전 흐름**에 따라 정리해 봅니다.

- **1) 통계 기반 벡터화 (BoW / TF-IDF)**
  딥러닝 이전의 방식인 **희소 표현(Sparse Representation)** 을 가볍게 짚어보고, 빈도 기반 방식의 한계를 이해합니다.

- **2) 신경망 기반의 분산 표현 (Word2Vec / Embedding Layer)**
  이 장의 핵심입니다. 단어의 의미를 **밀집 벡터(Dense Vector)**로 압축하여 학습하는 원리와 그 효과를 살펴봅니다.

- **3) 차세대 모델로의 연결 (Transformer)**
  고정된 임베딩을 넘어, 다음 장에서 배울 **BERT, GPT**가 문맥을 어떻게 반영하는지 간단히 연결합니다.

### 3.2 통계 기반 벡터화: BoW / TF-IDF / N-gram 

앞에서 본 것처럼, 우리가 진짜로 원하는 것은  
> 의미가 비슷한 단어는 벡터 공간에서도 가깝게,  
> 다른 의미의 단어는 멀리 떨어지게 만드는 **임베딩(embedding)** 입니다.

그런데 임베딩이 등장하기 전/초기에는,  
텍스트를 숫자로 표현하기 위해 **통계(statistics)에 기반한 벡터화 기법**들이 널리 사용되었습니다.

- 이 기법들은 **“임베딩”이라고 부르기보다는**,  
  “단어/문서의 등장 패턴을 숫자로 표현한 벡터화 방법”에 가깝습니다.
- 즉, **의미를 잘 담은 밀집 임베딩이라기보다는**,  
  “등장 횟수·빈도·조합”을 수치화한 **전통적인 표현 방식**이라고 보는 것이 더 정확합니다.

이 섹션에서는 대표적인 통계 기반 벡터화 기법인 **Bag of Words(BoW), TF-IDF, N-gram**을 간단히 살펴보고,  
이후 섹션에서 등장할 **신경망 기반 임베딩**과 대비해 보겠습니다.

#### 1) Bag of Words (BoW)

**아이디어**  
- 문서 안에 어떤 단어들이 **몇 번** 등장했는지만 세어서 벡터로 만드는 방식
- 단어의 **순서나 문맥은 완전히 무시**하고, “가방 속에 단어 개수만 넣어둔다”고 생각하면 됩니다.

  <img src="image/bow.webp" width="500">    
   
  이미지 출처 : [AIML.com Research](https://aiml.com/what-are-the-advantages-and-disadvantages-of-bag-of-words-model/)

```text
문장1: "고양이가 물을 마신다"
문장2: "물이 고양이를 마신다"

어휘 사전: ["고양이", "물", "마신다"]

→ 문장1 BoW: [1, 1, 1]
→ 문장2 BoW: [1, 1, 1]

두 문장의 단어 순서와 의미는 분명히 다르지만,
BoW 벡터는 완전히 동일하게 나옵니다.
```

- 장점:  
  - 구현이 단순하고 직관적
  - 전통적인 문서 분류, 스팸 필터링 등에서 오랫동안 잘 동작해 옴
- 한계:  
  - 단어 **순서, 문맥, 구문 구조**를 전혀 반영하지 못함  
  - 단어 사전 크기만큼 차원이 필요 → **고차원 희소 벡터(sparse vector)**

#### 2) TF-IDF (Term Frequency – Inverse Document Frequency)
TF-IDF는 정보 검색과 텍스트 마이닝에서 널리 쓰이는 단어의 상대적 중요도를 나타내는 통계적 가중치입니다.  
여러 문서(corpus) 중 특정 단어가 개별 문서에서 얼마나 중요한지를 나타내며,  
단순 빈도뿐 아니라 “문서 내 상대적 빈도”와 “전체 문서에서의 희소성”을 함께 고려합니다.

- BoW는 단순히 "단어가 몇 번 나왔는가"만 보지만,  
- TF-IDF는 "문서 내에서의 상대적 중요도 + 전체 문서에서의 희소성"을 함께 고려합니다.  

```
문서 집합에서:
"the"  → 거의 모든 문서에 자주 등장 → 중요도 낮음
"quantum" → 일부 문서에서만 드물게 등장 → 중요도 높음
```
<img src="image/tfidf.png">

#### 구성 요소

1. **TF (Term Frequency)**  
   - 특정 단어 *t*가 문서 *d* 안에서 얼마나 자주 등장했는지를 나타냄  
   - 단순히 "등장 횟수"를 쓰기도 하고, 문서 길이를 고려해 "등장 비율"로 정규화하기도 함  
   - 자주 나온 단어가 높은 점수  

   **공식**  
   $$
   TF(t, d) = \frac{\text{단어 t가 문서 d에 등장한 횟수}}{\text{문서 d의 전체 단어 수}}
   $$


2. **IDF (Inverse Document Frequency)**  
   - 특정 단어가 전체 문서 집합에서 얼마나 희귀한지를 나타냄  
   - 흔하게 나오는 단어는 IDF가 0에 가까워지고, 드물게 나오는 단어는 IDF가 커짐

   **공식**  
   $$
   IDF(t) = \log \frac{N(전체문서수)}{df(t)}
   $$

   - $ N $: 전체 문서 수  
   - $ df(t) $: 단어 t가 등장한 문서의 수(document frequency)  
   - 즉, 다른 문서에 거의 등장하지 않는 단어일수록 높은 점수



3. **TF-IDF**  
   - 최종적으로 TF와 IDF를 곱해 단어의 중요도를 산출  
   - 다른 문서에 자주 등장하지 않는 단어가 문서A에서 자주 등장했다면, 해당 단어는 문서 A에서 높은 TF-IDF 스코어 획득 

   **공식**  
   $$
   TF\text{-}IDF(t, d) = TF(t, d) \times IDF(t)
   $$

#### 계산 예시

문서 집합:
- 문서1: `"나는 고양이를 좋아한다"` (총 3단어)
- 문서2: `"나는 개를 좋아한다"` (총 3단어)  
전체 문서 수 $N=2$  
어휘: {"고양이", "개", "좋아한다", "나는"}

#### Step 1. IDF 계산 (단어의 공통 중요도)

| 단어 | 문서 빈도 (df, Document Frequency) | IDF 계산 (\log(N/df)) | 결과 (\log_{10} 사용) |
| :--- | :--- | :--- | :--- |
| **"고양이"** | 문서1에만 등장 $\rightarrow$ df=1 | $\log (2/1)$ | $\approx 0.301$ |
| **"개"** | 문서2에만 등장 $\rightarrow$ df=1 | $\log (2/1)$ | $\approx 0.301$ |
| **"좋아한다"** | 문서1, 문서2 모두 등장 $\rightarrow$ df=2 | $\log (2/2) = \log 1$ | $0$ |
| **"나는"** | 문서1, 문서2 모두 등장 $\rightarrow$ df=2 | $\log (2/2) = \log 1$ | $0$ |

#### Step 2. 문서1의 TF-IDF 벡터 계산 (문서1: "나는 고양이를 좋아한다")

| 단어 | TF (Term Frequency, 1/3) | IDF (Step 1 결과) | TF-IDF (TF $\times$ IDF) |
| :--- | :--- | :--- | :--- |
| **"고양이"** | $1/3 \approx 0.33$ | $0.301$ | $\approx 0.0993$ |
| **"개"** | $0$ | $0.301$ | $0$ |
| **"좋아한다"** | $1/3 \approx 0.33$ | $0$ | $0$ |
| **"나는"** | $1/3 \approx 0.33$ | $0$ | $0$ |
<br>

**결과: 문서1의 TF-IDF 벡터** $\approx [0.0993, 0, 0, 0]$ (어휘 순)

#### Step 3. 문서2의 TF-IDF 벡터 계산 (문서2: "나는 개를 좋아한다")

| 단어 | TF (Term Frequency, 1/3) | IDF (Step 1 결과) | TF-IDF (TF $\times$ IDF) |
| :--- | :--- | :--- | :--- |
| **"고양이"** | $0$ | $0.301$ | $0$ |
| **"개"** | $1/3 \approx 0.33$ | $0.301$ | $\approx 0.0993$ |
| **"좋아한다"** | $1/3 \approx 0.33$ | $0$ | $0$ |
| **"나는"** | $1/3 \approx 0.33$ | $0$ | $0$ |

**결과: 문서2의 TF-IDF 벡터** $\approx [0, 0.0993, 0, 0]$ (어휘 순)

**최종 결론:**
TF-IDF는 이 과정을 통해 다음을 달성합니다.
1.  **공통 단어("나는", "좋아한다")**: 두 문서에 흔하게 나타나므로 $\text{IDF}$가 $0$이 되어 **중요도 0**을 부여합니다.
2.  **핵심 단어("고양이", "개")**: 해당 문서에서만 특별하게 나타나므로 $\text{IDF}$가 높아져 **높은 중요도**를 부여합니다.

결과적으로, TF-IDF는  
문서1의 특징을 **고양이**로,  
문서2의 특징을 **개**로  
**명확하게 벡터 안에 표현**해 줍니다.

TF-IDF는 통계 기반 벡터화 방식 중 가장 널리 쓰였으며, 특히 다음과 같은 분야에서 핵심적인 역할을 수행했습니다.

1. **정보 검색 (Information Retrieval):**
- 사용자가 검색한 **쿼리(Query)의 중요도**를 계산하고, 웹 문서나 데이터베이스의 **문서 중요도**를 계산하는 데 사용되었습니다. 
- TF-IDF 값이 높은 문서일수록 사용자의 검색 의도와 가장 관련이 높다고 판단하는 방식입니다.
2. **문서 분류 및 클러스터링 (Clustering):**
- 뉴스 기사나 메일 같은 문서들을 TF-IDF 벡터로 변환한 후, 이 벡터들 간의 유사도(거리)를 계산하여   
비슷한 주제의 문서끼리 묶거나(클러스터링),  문서를 특정 주제로 분류하는(Classification)  
**특징 벡터(Feature Vector)** 로 사용되었습니다.

TF-IDF는 단순하지만 강력한 성능을 보여주며, 이후 **단어 중요도 측정** 방식의 기본 토대가 되었습니다.

#### 요약
- TF: 단어가 문서 내에서 얼마나 자주 등장했는가  
- IDF: 단어가 전체 문서 집합에서 얼마나 희귀한가  
- TF-IDF: "문서 내 중요도 × 전체 문서 희소성"  
- 장점: 문서 핵심 키워드 뽑기에 유용, 검색 엔진·문서 추천· 분류에 널리 활용  
- 한계: 여전히 단어 순서·문맥은 반영하지 못함  

#### TF-IDF 실습

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

corpus = [ "I love natural language processing",
          "I love programming in Python",
        "natural language processing is exciting",
        "Harry Potter is a series of seven fantasy fantasy novels"]

tfidf = TfidfVectorizer(stop_words='english').fit(corpus)

print(tfidf.vocabulary_)
pd.DataFrame(tfidf.transform(corpus).toarray(), columns=tfidf.get_feature_names_out())

#### 3) N-gram

BoW/TF-IDF는 단어의 **순서**를 완전히 무시한다는 큰 한계가 있습니다.  
이를 조금이라도 보완하기 위해 등장한 것이 **N-gram**입니다.

- 연속된 N개의 단어(또는 문자)를 하나의 묶음으로 보는 방법
- 예: 2-gram(바이그램), 3-gram(트라이그램)

```text
문장: "자연어 처리를 열심히 공부한다"

2-gram (바이그램):
["자연어 처리를", "처리를 열심히", "열심히 공부한다"]
```

- 장점:
  - 연속된 단어 패턴을 반영 → **문맥의 일부**를 표현 가능
  - 예: “뉴욕”, “인공 지능”, “자연어 처리” 같은 구를 잘 잡아낼 수 있음
- 한계:
  - N을 크게 할수록 경우의 수가 폭발 → 차원 수가 매우 커짐
  - 희귀하게 등장하는 N-gram은 데이터가 거의 없어 다루기 어려움
  - 여전히 **의미/유사도**를 직접적으로 학습한다기보다는, “공동 등장 패턴”에 가깝다

#### 4) 통계 기반 벡터화 정리

지금까지 본 BoW / TF-IDF / N-gram은

- 텍스트를 벡터로 만드는 **전통적인 통계 기반 표현 방식**이고,
- “벡터화”는 했지만, 우리가 3.1에서 정의한  
  **“의미를 잘 반영하는 밀집 임베딩”과는 거리가 있습니다.**

정리하자면,

- 장점:
  - 단순하고 구현이 쉽다
  - 문서 분류, 키워드 추출, 검색 등 고전적인 NLP 태스크에서 여전히 유용하다
- 근본적인 한계:
  - 단어 순서, 문맥, 깊은 의미를 잘 반영하지 못한다
  - 고차원 희소 벡터가 되어 계산·메모리 효율이 떨어진다
  - 비슷한 의미의 단어를 **벡터 공간에서 가깝게 모으는 구조가 아니다**

이런 이유로,  
> “단어/문장의 **의미**를 벡터 공간에 잘 담아내고 싶다”는 요구가 커지면서  
> **신경망 기반의 임베딩(Embedding Layer, Word2Vec, FastText 등)** 이 본격적으로 사용되기 시작했습니다.

다음 섹션에서는 딥러닝 모델 안에서 사용하는  
**임베딩 레이어(Embedding Layer)** 가 어떻게 동작하는지부터 살펴보겠습니다.

### 3.3 신경망 기반 임베딩: 임베딩 레이어

앞에서 본 BoW / TF-IDF / N-gram은  
- “단어가 얼마나 / 어떻게 등장했는지”에 초점을 맞춘 **통계 기반 벡터화**였습니다.

이제부터는 딥러닝 모델 안에서 사용되는  
**신경망 기반 임베딩(Embedding Layer)** 을 살펴봅니다.

#### 2) 임베딩 레이어의 학습

임베딩 레이어의 가중치(테이블의 각 행)는  
모델 학습 과정에서 **다른 레이어와 함께 업데이트**됩니다.

- 처음에는 임베딩 테이블이 **무작위 값**으로 초기화됩니다.
- 모델이 학습 데이터에 대해 **손실(loss)** 을 계산하고,
- 역전파(backpropagation)를 통해  
  손실을 줄이는 방향으로 임베딩 벡터들이 조금씩 조정됩니다.
- 이 과정이 반복되면서,
  - 비슷한 문맥에서 자주 쓰이는 단어들의 벡터는 비슷한 방향으로 모이고,
  - 역할이 다른 단어들은 서로 다른 방향으로 흩어지게 됩니다.

결국, 임베딩 레이어의 가중치 행렬 전체가  
> 학습 데이터로부터 얻은 **단어 의미 공간(embedding space)**

이라고 볼 수 있습니다.

#### 3) 간단한 코드 예시 (PyTorch)

아래는 아주 작은 예시입니다.

```python
import torch
import torch.nn as nn

# 예시: 단어 사전 크기 10, 임베딩 차원 4
vocab_size = 10
embedding_dim = 4

embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

# 예시 문장: 단어 ID 시퀀스 (batch_size=2, sequence_length=3)
# 문장1: [1, 2, 3]
# 문장2: [2, 5, 7]
input_ids = torch.tensor([[1, 2, 3],
                          [2, 5, 7]])  # shape: (2, 3)

# 임베딩 레이어 통과
emb = embedding(input_ids)  # shape: (2, 3, 4)

print("입력 ID shape:", input_ids.shape)
print("임베딩 결과 shape:", emb.shape)
```

출력 형태를 보면,

- 입력: `(batch_size, sequence_length)`  
- 출력: `(batch_size, sequence_length, embedding_dim)`

이 됩니다.  
이제 이 임베딩 벡터 시퀀스가 RNN, LSTM, Transformer 등의 **다음 레이어 입력**으로 들어가게 됩니다.

#### 4) 임베딩 레이어 vs 통계 기반 벡터화

- BoW / TF-IDF / N-gram:
  - “규칙”으로 벡터를 만드는 방식  
  - 벡터 값이 **고정** (데이터를 더 봐도 바뀌지 않음)
  - 고차원 희소 벡터가 되기 쉽고, 의미/유사도를 직접 학습하지는 않음

- 임베딩 레이어:
  - **모델 학습 과정에서 벡터가 함께 학습**됨  
  - 데이터에 맞춰 **단어 의미, 문맥 정보**를 벡터에 녹여낼 수 있음
  - 비교적 낮은 차원의 **밀집(dense) 벡터**

그래서 임베딩 레이어는
> “딥러닝 모델 안에서 자동으로 학습되는 단어 임베딩 테이블”

### 3.4 분산 표현: Word2Vec / FastText

임베딩 레이어는 **딥러닝 모델 안에 포함된 임베딩 테이블**이었습니다.  
이번에는 임베딩만 따로 학습하는 대표적인 신경망 기법인  
**Word2Vec**과 **FastText**를 살펴봅니다.

#### 1) 분산 표현(Distributed Representation)이란?

기존 통계 기반 벡터화(BoW, TF-IDF)는

- 단어마다 “등장 횟수/빈도/조합”을 세어 벡터로 만들었습니다.
- 단어 의미나 문맥을 직접 학습한다기보다, **통계를 정리한 결과**에 가깝습니다.

반면, 분산 표현은

> “비슷한 문맥에서 쓰이는 단어는 비슷한 벡터를 갖도록”  
> 신경망으로 **벡터 자체를 학습**하는 접근입니다.  
> (의미가 여러 차원에 분산되어 있다는 뜻. 원핫은 오직 한 차원('1'이 있는 그 위치)에만 정보가 있음)

- 각 단어는 보통 수십~수백 차원의 **밀집 벡터(dense vector)** 로 표현되고,
- 벡터 공간에서의 위치/거리/각도가 의미 정보를 반영하도록 학습됩니다.

#### 2) Word2Vec

Word2Vec은 **아주 얕은 신경망**을 사용해 단어 임베딩을 학습하는 모델입니다.

핵심 아이디어:

- 단어는 **주변 단어(문맥)** 과 함께 나타납니다.
- “비슷한 문맥에서 등장하는 단어들끼리는 의미가 비슷할 가능성이 크다.”
- 따라서, 문맥 예시들을 많이 보여주면서  
  임베딩이 그 패턴을 잘 예측하도록 학습합니다.

주요 구조는 두 가지입니다.

- **CBOW (Continuous Bag of Words)**  
  - 주변 단어들(컨텍스트) → 중심 단어를 예측  
  - 예: `"나는 ___을 먹었다"` → 빈칸에 들어갈 단어를 맞히도록 학습
- **Skip-Gram**  
    <img src="image/word2vec-skip-gram.png" width="600">  
    이미지 출처 : https://lilianweng.github.io/posts/2017-10-15-word-embedding/
  - 중심 단어 → 주변 단어들을 예측  
  - 예: `"밥"` → (나는, 먹었다) 같은 주변 단어들을 맞히도록 학습

학습이 잘 되면, 임베딩 벡터 공간에서  
재미있는 **벡터 연산**이 가능해지는 것으로 유명합니다.

```text
벡터("왕") - 벡터("남자") + 벡터("여자") ≈ 벡터("여왕")
```

즉, 단어들 사이의 관계(성별, 직업, 국가 등)가  
벡터 공간에서 **산술 연산**으로도 어느 정도 드러납니다. 

#### 3) FastText

FastText는 Facebook에서 제안한, Word2Vec의 확장 모델입니다.

- Word2Vec: 단어 하나를 하나의 토큰으로 보고, 그에 대한 임베딩을 학습
- FastText: 단어를 더 잘게 쪼갠 **subword(부분 문자열)** 단위 임베딩을 학습

예를 들어,

단어: "자연어처리"

subword (문자 n-gram 예시):
"<자연", "자연어", "연어처", "처리>", ... (시작/끝 토큰 포함)

- 각 subword에 대한 임베딩을 학습한 후,
- 이를 합쳐서 단어 전체의 임베딩을 구성합니다.

장점:

- **희귀 단어, 오탈자, 신조어** 에도 비교적 잘 대응  
  - 단어 전체가 학습에 한 번도 등장하지 않았더라도, 그 안에 포함된 subword들이 학습되어 있으면  
    벡터를 어느 정도 추론해서 만들 수 있습니다.  
- 한국어처럼 합성어나 신조어가 많은 언어에서도 유리한 점이 있습니다.

#### Word2Vec 대비 강점

Word2Vec은 혁신적인 모델이었으나, 실제 자연어 처리 서비스를 구축할 때는 몇 가지 치명적인 한계가 있었습니다.  
 FastText는 단어를 더 잘게 쪼개는 '서브워드(Subword)' 방식을 도입하여 이를 해결합니다.

**1. 미등록 단어(OOV, Out-of-Vocabulary)의 정교한 처리**
- **Word2Vec:** 학습 데이터에 없던 단어(예: 신조어 '멍뭉이')가 입력되면, 단어 전체를 '모르는 단어(`<UNK>`)'로 처리하여 아무런 의미를 파악하지 못합니다.
- **FastText:** '멍뭉이'를 처음 보더라도 그 안의 '멍', '뭉', '이' 같은 글자 조합(Subword)을 추출합니다. 기존에 배웠던 '강아지'나 '댕댕이'와 공유하는 조각들이 있다면, 이를 통해 단어의 의미 벡터를 유추해낼 수 있습니다.

**2. 형태학적 유사성(Morphological Richness) 반영**
- **Word2Vec:** '먹다', '먹었다', '먹을까'를 완전히 다른 단어로 인식합니다. (학습 데이터가 부족하면 세 단어 사이의 연관성을 찾지 못함)
- **FastText:** 모든 단어 안에 공통으로 포함된 '먹'이라는 조각을 인식합니다. 덕분에 어미가 변해도 단어의 뿌리(어근)가 가진 핵심 의미를 벡터 공간 안에 강력하게 보존합니다.

**3. 입력 데이터의 불완전함(오탈자)에 대한 강인함**
- 서비스 중 사용자가 '자연어'를 '자연허'라고 오타를 냈을 때, FastText는 '자연', '연허' 등의 조각 정보를 활용해 원래 의미인 '자연어'와 매우 유사한 벡터를 생성합니다.
- **활용 예시:**
    - **내부 검색 로직:** 쇼핑몰이나 앱 내 검색에서 사용자가 오타를 내도 비슷한 상품을 추천해 주는 **'의미 기반 매칭'** 부품으로 활용됩니다.
    - **챗봇/상담 시스템:** "배송 언제와ㅡㅡ"와 같이 변칙적인 입력이 들어와도 문장의 의도를 놓치지 않고 파악하는 **'전처리/임베딩'** 단계에서 강력한 힘을 발휘합니다.

**정리:** FastText는 거대 검색 엔진 그 자체는 아니지만, **사용자의 불완전한 입력(오타, 신조어, 어미 변형)을 기계가 '이해 가능한 의미'로 변환해 주는 핵심 부품**으로서 실무에서 매우 높은 가치를 가집니다. 

#### 4) Word2Vec / FastText vs 임베딩 레이어

- 공통점:
  - 둘 다 “단어를 **의미 벡터**로 표현”하려는 시도
  - 학습 데이터의 문맥 정보를 이용해 벡터를 학습

- 차이점:
  - Word2Vec / FastText:
    - **독립된 모델**로 임베딩만 미리 학습 (pre-train)  
    - 이후 다양한 다운스트림 태스크에 이 임베딩을 가져다 쓸 수 있음
  - 임베딩 레이어:
    - 특정 태스크용 딥러닝 모델 안에 포함되어  
      **태스크와 함께 end-to-end로 학습**됨

실제 프로젝트에서는

- 미리 학습된 Word2Vec/FastText 임베딩을  
  임베딩 레이어 초기값으로 불러와 사용하는 방식도 많이 씁니다.

### 3.5 현대 NLP에서의 임베딩: "고정된 벡터"에서 "계산되는 표현"으로

우리가 지금까지 배운 Word2Vec이나 FastText는 단어마다 하나의 고유한 벡터 좌표를 할당하는 **정적(Static) 임베딩**이었습니다.  
하지만 현대 NLP는 여기서 한 단계 더 진보하여, 단어의 벡터를 문장 속에서 실시간으로 만들어내는 **문맥적 임베딩(Contextual Embedding)** 시대로 넘어왔습니다.

#### 1) 작동 방식의 변화: "찾아보기"에서 "실시간 합성"으로

가장 큰 차이는 임베딩이 결정되는 **시점**과 **방식**에 있습니다.

- **기존 방식 (Word2Vec/FastText):** - 메모리에 저장된 **임베딩 표(Lookup Table)** 에서 단어에 맞는 벡터를 꺼내오면 끝입니다.
- 예를 들어 "은행"은 어떤 문장에서도 항상 `[0.1, -0.5]` 같은 고정된 값입니다.
- **현대 방식 (Transformer/BERT 등):** - 임베딩 레이어에서 단어의 기본 벡터를 가져온 뒤,  
**신경망 층(Encoder)** 을 통과하며 주변 단어들의 정보를 수치적으로 **합산(Combine)** 합니다.
- 예를 들어, `"사과를 먹다"`라는 문장에서 '사과' 벡터는 옆에 있는 '먹다'라는 단어의 정보를 흡수하여  
**음식**의 특징이 실시간으로 강화된 새로운 벡터로 변신합니다.

#### 2) 학습 메커니즘의 진화: 빈칸 채우기(Masking)를 통한 문맥 파악

현대 모델은 단순히 단어의 짝을 맞히는 것을 넘어, 문장 전체의 구조를 이해하기 위해 훨씬 고차원적인 방식으로 사전 학습(Pre-training)됩니다.

- **마스크 학습 (Masked Language Modeling):**
  - 문장 중간의 단어를 가려놓고(`[MASK]`), 주변 단어들을 조합해 그 빈칸에 들어갈 단어를 맞히는 연습을 합니다.
  - 예: `"나는 오늘 [MASK]에 가서 돈을 입금했다"` → 주변의 '돈', '입금'을 보고 `[MASK]`가 '은행'임을 추론해야 합니다.
- **문맥 정보의 내재화:** 
  - 이 빈칸을 맞히기 위해 모델은 **"주변에 어떤 단어가 올 때 이 단어의 의미가 어떻게 변하는지"** 그 관계(Attention)를 아주 정교하게 학습하게 됩니다.
  - 이 과정이 수억 개의 문장에 대해 반복되면서, 단어 벡터는 단순한 의미를 넘어 **문맥을 읽어내는 능력**을 갖추게 됩니다.

#### 3) 정적 임베딩 vs 문맥적 임베딩 비교 요약

| 구분 | 정적 임베딩 (Word2Vec, FastText) | 문맥적 임베딩 (BERT, GPT 등) |
| :--- | :--- | :--- |
| **핵심 원리** | 미리 저장된 벡터를 **꺼내오기(Lookup)** | 주변 단어와 정보를 **섞어서 계산(Compute)** |
| **다의어 처리** | "은행"의 모든 의미가 하나의 벡터에 섞임 | 문맥(주변 단어)을 보고 실시간으로 의미 분리 |
| **학습 방식** | 단어 간의 단순 통계적 유사성 학습 | **빈칸 채우기(Masking)** 등으로 문맥 간 관계 학습 |
| **구성 요소** | 단어(또는 서브워드)의 고유 의미 | **토큰 의미 + 순서(Position) + 문맥 정보** |

**정리:** 현대 NLP 임베딩의 핵심은 **학습 시점에 빈칸 채우기 등을 통해 단어들 사이의 고차원적인 관계를 미리 파악하고,  
이를 바탕으로 실제 문장에서 단어의 의미를 실시간으로 계산해낸다**는 점에 있습니다.

이러한 '실시간 계산 엔진'의 정체인 **Transformer**와 **Attention**의 구조를 다음 장에서 본격적으로 파헤쳐 보겠습니다.

#### 정리
- **원-핫 인코딩**: 단어 구분만 가능, 유사성 반영 불가  
- **통계 기반 임베딩**: 빈도·중요도·순서를 반영하지만, 의미는 여전히 부족  
- **분산 표현(Word2Vec, FastText)**: 신경망을 통해 **단어 간 의미 관계**까지 수치화  
- → 이는 이후 **더 복잡한 딥러닝 NLP 모델**(예: RNN, Transformer)의 기초 표현으로 활용됨 

#### 문제 10. TF-IDF
문장 리스트 `["나는 NLP를 좋아한다", "NLP는 재미있다"]`를 TF-IDF 벡터로 변환하세요.  

<details>
<summary>정답 보기</summary>

```python
from sklearn.feature_extraction.text import TfidfVectorizer

docs = ["나는 NLP를 좋아한다", "NLP는 재미있다"]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(docs)

print(vectorizer.get_feature_names_out())
print(X.toarray())
```
</details>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

docs = ["나는 NLP를 좋아한다", "NLP는 재미있다"]

#### 문제 11. BoW 벡터 만들기
문장 리스트 `["나는 밥을 먹었다", "나는 물을 마셨다"]`에서 **Bag of Words** 벡터를 만들어보세요.  

<details> <summary>정답 보기</summary>

```python
from sklearn.feature_extraction.text import CountVectorizer

docs = ["나는 밥을 먹었다", "나는 물을 마셨다"]
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(docs)

print("단어 사전:", vectorizer.get_feature_names_out())
print("BoW 벡터:\n", X.toarray())
```
</details>

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

docs = ["나는 밥을 먹었다", "나는 물을 마셨다"]

#### 문제 12. TF-IDF 계산
문장 리스트 `["나는 NLP를 공부한다", "NLP는 재미있다", "공부는 힘들다"]`를 **TF-IDF**로 벡터화하세요.  

<details> <summary>정답 보기</summary>

```python
from sklearn.feature_extraction.text import TfidfVectorizer

docs = ["나는 NLP를 공부한다", "NLP는 재미있다", "공부는 힘들다"]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(docs)

print("단어 사전:", vectorizer.get_feature_names_out())
print("TF-IDF 벡터:\n", X.toarray())
```
</details>

In [ ]:
# 여기에 작성하세요
from sklearn.feature_extraction.text import TfidfVectorizer

docs = ["나는 NLP를 공부한다", "NLP는 재미있다", "공부는 힘들다"]

#### 문제 13. N-gram 적용
문장 `"자연어 처리를 공부한다"`에 대해 **2-gram(바이그램)** 을 추출해보세요.  

<details> <summary>정답 보기</summary>

```python
from sklearn.feature_extraction.text import CountVectorizer

docs = ["자연어 처리를 공부한다"]
vectorizer = CountVectorizer(ngram_range=(2,2))
X = vectorizer.fit_transform(docs)

print("바이그램 사전:", vectorizer.get_feature_names_out())
print("바이그램 벡터:", X.toarray())
```
</details>


In [ ]:
# 여기에 작성하세요
from sklearn.feature_extraction.text import CountVectorizer

docs = ["자연어 처리를 공부한다"]

#### 문제 14. Word2Vec 학습
문장 리스트 `["나는 학교에 간다", "학교에서 공부한다", "나는 밥을 먹었다"]`로 Word2Vec 모델을 학습한 뒤, `"학교"`와 가장 유사한 단어를 찾아보세요.  

<details> <summary>정답 보기</summary>

```python
from gensim.models import Word2Vec

sentences = [
    ["나는", "학교에", "간다"],
    ["학교에서", "공부한다"],
    ["나는", "밥을", "먹었다"],
    ["좋은", "학교", "수업"],
    ["맛있는", "밥"],
    ["시원한", "콜라를","마셨다"],
]

model = Word2Vec(sentences, vector_size=50, window=3, min_count=1, sg=0)

print("학교와 가장 유사한 단어:", model.wv.most_similar("학교", topn=1))
```
</details>

In [ ]:
# 여기에 작성하세요
from gensim.models import Word2Vec

sentences = [
    ["나는", "학교에", "간다"],
    ["학교에서", "공부한다"],
    ["나는", "밥을", "먹었다"],
    ["좋은", "학교", "수업"],
    ["맛있는", "밥"],
    ["시원한", "콜라를","마셨다"],
]

#### 문제 15. Word2Vec 벡터 연산
Word2Vec을 학습한 모델에서 `"밥"` - `"먹었다"` + `"마셨다"` ≈ ? 를 계산해보세요.  

<details> <summary>정답 보기</summary>

```python
print(model.wv.most_similar(positive=["밥", "마셨다"], negative=["먹었다"], topn=1))
```
</details>


In [ ]:
# 여기에 작성하세요


#### 문제 16. FastText 희귀 단어 처리
FastText 모델을 학습시킨 뒤, 사전에 없는 `"자연어처리학개론"`을 입력했을 때 벡터가 생성되는지 확인하세요.  

<details> <summary>정답 보기</summary>

```python
from gensim.models import FastText

sentences = [
    ["나는", "자연어", "처리", "공부한다"],
    ["FastText", "모델", "테스트"]
]

model = FastText(sentences, vector_size=50, window=3, min_count=1)

print("임베딩 벡터 크기:", model.wv["자연어처리학개론"].shape)
# FastText는 subword 기반이므로 학습에 없던 단어도 벡터 생성 가능
```
</details>

In [ ]:
# 여기에 작성하세요


## 4. 순환 신경망(Recurrent Neural Network, RNN)

<img src="image/rnn_hidden_state.png" width=500>

### 4.1 기본 개념
- **순차적 데이터를 처리하기 위한 신경망**: 순환 신경망은 시퀀스 데이터를 처리하도록 설계된 인공신경망의 한 종류로, 과거 정보를 기억하면서 새로운 정보를 처리하는 모델입니다.  
- **은닉 상태(hidden state)를 사용하여 과거 정보를 기억**: RNN은 각 단계에서 이전 단계의 정보를 은닉 상태로 저장하고 이를 다음 단계로 전달하여, 주식 가격, 날씨, 자연어 처리와 같이 순차적 맥락이 중요한 문제에서 뛰어난 성능을 발휘합니다.  

- **텍스트를 한 단어씩 읽으면서 문맥(Context)을 반영**: 예를 들어, 문장이나 음성 데이터를 순차적으로 처리하며 시점 간의 연관성을 파악합니다.  
- **순환 구조와 파라미터 공유**: RNN은 내부 순환 구조를 통해 이전 단계의 출력을 현재 단계의 입력으로 사용하며, 시간에 따라 파라미터를 공유해 효율적으로 학습합니다.  

  <img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FIe6mK%2FbtrrYR2JYzE%2FYBHMBjxLskjI7zETkmgrX0%2Fimg.png" width="600">

#### 1) RNN의 핵심 가중치와 역할

RNN은 매 시점마다 다음 세 가지 가중치를 사용하여 정보를 처리합니다.

* **$W_{xh}$ (입력 가중치):** 현재 들어온 데이터($x_t$)에서 어떤 정보를 추출해 은닉 상태로 보낼지 결정합니다.  
입력을 모델의 내부 표현(Hidden) 크기로 변환하는 첫 번째 관문입니다.
* **$W_{hh}$ (순환 가중치):** 이전의 기억($h_{t-1}$)을 얼마나 유지하고 반영할지 결정합니다.  
이 행렬 덕분에 RNN은 과거의 정보를 현재로 전달하는 '시간의 흐름'을 가집니다.
* **$W_{hy}$ (출력 가중치):** 최종 정리된 은닉 상태($h_t$)를 바탕으로 우리가 보고 싶은 형태(예: 다음 단어 확률 등)의 결과물($y_t$)을 만들어냅니다.

#### 2) Shape과 사이즈 (은닉 상태가 32인 경우)

데이터가 1개인 경우(Batch size = 1)를 가정하면, 각 행렬은 **차원을 맞추어 정보를 통합**하기 위해 다음과 같은 크기를 가집니다.

| 구성 요소 | Shape (Size) | 연산의 의미 |
| :--- | :--- | :--- |
| **$h_{t-1}$** | $32 \times 1$ | 이전 시점에서 넘어온 32차원의 '기억' 데이터 |
| **$W_{hh}$** | **$32 \times 32$** | 기억($32$)을 다시 기억($32$)으로 변환하며 관계를 학습 |
| **$x_t$** | $d \times \text{1}$ | 현재 입력된 임베딩 벡터 (예: $d=128$) |
| **$W_{xh}$** | **$\text{32} \times d$** | 입력($128$)을 은닉 상태 크기($32$)로 압축/변환 |
| **$b$** | **$32 \times 1$** | 합산된 32개 노드 각각에 더해지는 고유한 편향값 |
| **$h_t$** | **$32 \times 1$** | **과거와 현재가 통합되어 새롭게 탄생한 32차원의 기억** |

> **핵심:** 서로 다른 크기에서 출발한 **현재 입력($W_{xh}x_t$)** 과 **이전 기억($W_{hh}h_{t-1}$)** 은  
가중치 행렬과의 곱셈을 통해 모두 **$32 \times 1$** 이라는 동일한 크기로 변신합니다.  
이렇게 규격이 같아졌기에 비로소 두 정보를 **'더해서'** 하나로 합칠 수 있게 되는 것입니다.

<img src="image/rnn_sequence.png" width=600>  

<br>

<img src="image/text_time_sequence.png">

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FcffBNq%2FbtrrXqRKRQM%2FZDsokUIc5RRUdFhAJT40jK%2Fimg.png">

**다양한 입출력 구조**: RNN은 데이터의 입력과 출력 형태에 따라 세 가지 주요 유형으로 나뉩니다:  
  1. **One-to-Many**: 하나의 입력에서 여러 출력을 생성. 
  - 예: 이미지 설명 생성(image captioning)에서 이미지 한 장을 입력받아 여러 단어로 이루어진 설명을 출력.  
  2. **Many-to-One**: 여러 입력 시퀀스를 받아 단일 출력을 생성 (입력은 여러개 출력은 하나). 
  - 예: 감정 분석(sentiment analysis)에서 문장의 단어 시퀀스를 처리해 긍정/부정 레이블을 출력.  
  3. **Many-to-Many**: 입력 시퀀스에 대해 출력 시퀀스를 생성. 
  - 예: 기계 번역(machine translation)에서 영어 문장을 입력받아 다른 언어의 문장을 출력.  

### 4.2 장단점
- **장점**:  
  - **순서 정보 보존**: RNN은 시퀀스 데이터의 순서를 고려하여 시간적 또는 순차적 패턴을 학습하는 데 적합합니다.  
  - **다양한 시퀀스 데이터 처리**: 텍스트, 음성, 동영상 등 연속된 데이터를 효과적으로 처리하며, 문맥을 반영해 패턴을 학습합니다.  
  - **파라미터 공유**: 시간에 따라 동일한 파라미터를 공유하므로, 시퀀스 길이가 길어져도 모델 크기가 효율적으로 유지됩니다.  

- **단점**:  
  - **긴 문맥 처리 어려움**: 긴 시퀀스에서 정보가 멀리 전달될 때 기울기 소실 문제(Vanishing Gradient Problem)로 인해 학습이 어려워질 수 있습니다.  
  - **한계 보완**: 긴 시퀀스 처리의 한계를 극복하기 위해 LSTM(Long Short-Term Memory)나 GRU(Gated Recurrent Unit)와 같은 발전된 모델이 사용됩니다.

### 4.3 다른 신경망과의 차이점
- 전방향 신경망(FNN)은 각 입력을 독립적으로 처리해 시간적 의존성을 다루기 어렵습니다.  
- CNN은 공간적 패턴(이미지 등)에 특화되어 있어 시계열 데이터 처리에는 한계가 있습니다.  
- RNN은 이전 단계 출력을 다음 단계 입력으로 넘기는 순환 구조로, 시퀀스의 순서를 반영해 학습합니다.

### 4.4 RNN 실습

In [ ]:
# # 필요한 라이브러리 import
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# 하이퍼파라미터 설정
max_features = 10000  # 사용할 단어의 최대 개수
max_len = 300         # 시퀀스 길이를 최대 500단어로 제한
embedding_dim = 128    # 임베딩 차원
batch_size = 64
epochs = 30
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:

# 1) IMDB 데이터 로드
# imdb_int_sequences.npz 파일 안에는 이미 전처리된 텍스트 데이터가 들어있다.
# x_train, x_test에는 리뷰가 정수 ID 시퀀스로 변환된 상태로 저장됨.
# y_train, y_test에는 긍정(1) / 부정(0) 라벨이 저장됨.
# allow_pickle=True는 가변 길이 리스트(np.array(object))를 로드하기 위해 필요하다.
data = np.load("data/imdb_int_sequences.npz", allow_pickle=True)

# 학습용/테스트용 입력 시퀀스와 라벨 분리
x_train, y_train = data["x_train"], data["y_train"]
x_test, y_test = data["x_test"], data["y_test"]

In [ ]:
# 2) pad_sequences: 서로 길이가 다른 문장을 같은 길이로 맞춰주는 함수
# 딥러닝 모델은 "고정 길이 입력"이 필요하므로 패딩 처리 필수
def pad_sequences(sequences, maxlen, padding="post", truncating="post", value=0):
    """
    sequences: 리뷰(문장)를 구성하는 정수 리스트들의 집합
    maxlen: 최종 문장의 길이(예: 500)
    padding: 'post'면 뒤에 0을 채우고, 'pre'면 앞을 0으로 채움
    truncating: 문장이 너무 길면 'post'는 뒤 자르기, 'pre'는 앞 자르기
    value: 패딩할 때 채울 값(보통 0)
    """

    n_samples = len(sequences)

    # (샘플 수, maxlen) 모양의 배열을 value(0)으로 채워 초기화
    # 아직은 모든 문장이 0으로 채워진 상태
    x = np.full((n_samples, maxlen), value, dtype=np.int64)

    # 각 문장을 maxlen 길이에 맞춰 잘라 넣기
    for i, seq in enumerate(sequences):

        # ① 길이가 너무 길다면 잘라내기
        if truncating == "pre":
            seq = seq[-maxlen:]     # 앞을 자르고 뒤쪽 maxlen만 사용
        else:  # truncating == "post"
            seq = seq[:maxlen]      # 뒤를 잘라 앞쪽 maxlen만 사용

        # ② 길이가 짧다면 패딩 채우기
        # 'pre'는 앞쪽을 0으로 채우고, 'post'는 뒤쪽을 0으로 채움
        if padding == "pre":
            x[i, -len(seq):] = seq
        else:  # padding == "post"
            x[i, :len(seq)] = seq

    return x

In [ ]:
# 3) 훈련/테스트 데이터를 동일한 길이로 패딩 처리
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

In [ ]:
# 4) 넘파이 배열 → PyTorch 텐서로 변환
# 입력 시퀀스는 정수 ID이므로 torch.long 타입 사용
# 라벨(y)은 BCELoss나 BCEWithLogitsLoss를 쓰므로 float 타입 사용
x_train_tensor = torch.tensor(x_train, dtype=torch.long)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


# 5) TensorDataset으로 “입력 + 라벨”을 하나의 데이터셋 형태로 묶기
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)


# 6) DataLoader로 배치(batch) 단위로 데이터 공급하기
# shuffle=True ⇢ 모델이 패턴을 외우지 않도록 매 epoch마다 순서를 섞어줌
# batch_size만큼 데이터를 잘라서 제공하므로 GPU 사용 시 매우 효율적
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [ ]:
# SimpleRNNModel: 기본적인 RNN을 사용한 감성분석 모델
class SimpleRNNModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden1, max_len):
        super(SimpleRNNModel, self).__init__()

        # 1) Embedding Layer
        # 각 단어(정수 ID)를 밀집 벡터(dense vector)로 변환하는 층.
        # 예: 단어 ID 15 → 64차원 벡터 [0.2, -0.1, ...]
        # vocab_size : 사용할 단어 사전 크기
        # embed_dim  : 단어를 몇 차원 벡터로 바꿀지
        self.embedding = nn.Embedding(vocab_size, embed_dim)

        # 2) RNN Layer (vanilla RNN)
        # 입력 시퀀스(문장)를 순차적으로 읽어가며 hidden state를 업데이트하는 층.
        # embed_dim : 입력 특징 차원
        # hidden1   : RNN 내부에서 사용할 hidden state 크기
        # batch_first=True → 입력 형태를 (batch, seq_len, features)로 받겠다는 의미
        # ※ 여기서는 단일 RNN 층(single-layer RNN)만 사용
        self.rnn1 = nn.RNN(embed_dim, hidden1, batch_first=True)

        # 3) Dropout Layer
        # 과적합(overfitting)을 방지하기 위해 일부 뉴런을 랜덤하게 0으로 만드는 기능.
        # 훈련 중에는 활성화되고, 평가/추론 시에는 자동으로 꺼짐.
        self.dropout1 = nn.Dropout(0.3)

        # 4) Fully Connected Layer (출력층)
        # 마지막 hidden state를 받아 감성(긍정/부정) 점수 1개를 출력.
        # BCEWithLogitsLoss를 쓰므로 sigmoid는 여기서 적용하지 않음!
        self.fc = nn.Linear(hidden1, 1)

    def forward(self, x):
        # x는 (batch, sequence_length) 형태의 정수 ID 입력

        # 1) 단어 ID → 임베딩 벡터 변환
        x = self.embedding(x)  # (batch, seq_len, embed_dim)

        # 2) RNN 층에 입력 → 모든 타임스텝 hidden state 출력
        out1, _ = self.rnn1(x)  # out1 shape: (batch, seq_len, hidden1)

        # 3) Dropout 적용 (훈련 중에만 활성)
        out1 = self.dropout1(out1)

        # 4) 마지막 타임스텝의 hidden state만 사용
        #    return_sequences=False 역할을 직접 구현한 것과 동일함.
        last_output = out1[:, -1, :]  # (batch, hidden1)

        # 5) Fully Connected Layer → 로짓(logit) 출력
        out = self.fc(last_output)    # (batch, 1)

        # Sigmoid는 적용하지 않음 (BCEWithLogitsLoss가 내부에서 자동 적용)
        return out


# 실제 모델 생성
# vocab_size     : 단어 사전 크기 (IMDB 기준 10,000)
# embedding_dim  : 단어 벡터 차원
# hidden1        : RNN hidden state 차원 (128로 설정)
# max_len        : 문장 최대 길이
model = SimpleRNNModel(max_features, embedding_dim, 128, max_len).to(device)
print(model)

#### 참고: RNN 계열 레이어의 출력 형태 (PyTorch)

PyTorch의 `nn.RNN`, `nn.LSTM`, `nn.GRU`는 항상 **전체 시퀀스의 출력**을 반환합니다.

- `output`: `(batch, seq_len, hidden_size)` 또는 `(seq_len, batch, hidden_size)`  
- `h_n` (그리고 LSTM의 경우 `c_n`): 마지막 타임스텝의 hidden state

문장 분류처럼 **마지막 타임스텝만 필요**할 때는,
```python
last_output = output[:, -1, :]  # (batch, hidden_size)

In [ ]:
# 손실 함수: BCEWithLogitsLoss는 sigmoid + BCE를 안정적으로 계산
criterion = nn.BCEWithLogitsLoss()

# Adam 최적화 알고리즘 사용 (lr=0.001)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습·검증 결과 저장 리스트
train_acc_list, val_acc_list = [], []
train_loss_list, val_loss_list = [], []

for epoch in tqdm(range(epochs)):
    model.train()  # 모델을 학습 모드로 설정 (dropout 활성)
    running_loss, correct, total = 0.0, 0, 0

    # 훈련 루프
    for x_batch, y_batch in train_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)  # GPU/CPU로 이동

        optimizer.zero_grad()  # 이전 gradient 초기화
        outputs = model(x_batch).squeeze()  # 모델 예측 (logits)
        loss = criterion(outputs, y_batch)  # 손실 계산
        loss.backward()  # 역전파
        optimizer.step()  # 파라미터 업데이트

        running_loss += loss.item() * x_batch.size(0)  # 배치 손실 합산
        preds = (outputs >= 0.5).float()  # 0.5 기준 이진 분류
        correct += (preds == y_batch).sum().item()  # 정확한 개수 누적
        total += y_batch.size(0)  # 데이터 개수 누적

    train_loss = running_loss / total  # 평균 훈련 손실
    train_acc = correct / total  # 훈련 정확도

    # 검증 루프
    model.eval()  # 평가 모드 (dropout 비활성)
    val_loss, val_correct, val_total = 0.0, 0, 0

    with torch.no_grad():  # 평가에서는 gradient 계산 X (메모리 절약)
        for x_batch, y_batch in test_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            outputs = model(x_batch).squeeze()
            loss = criterion(outputs, y_batch)
            val_loss += loss.item() * x_batch.size(0)

            preds = (outputs >= 0.5).float()
            val_correct += (preds == y_batch).sum().item()
            val_total += y_batch.size(0)

    val_loss /= val_total  # 평균 검증 손실
    val_acc = val_correct / val_total  # 검증 정확도

    # 결과 기록
    train_acc_list.append(train_acc)
    val_acc_list.append(val_acc)
    train_loss_list.append(train_loss)
    val_loss_list.append(val_loss)

    # epoch별 결과 출력
    print(f"Epoch [{epoch+1}/{epochs}] "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

In [ ]:
# 정확도/손실 시각화
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(range(1, epochs + 1), train_acc_list, label='Training Accuracy')   # 훈련 정확도
plt.plot(range(1, epochs + 1), val_acc_list, label='Validation Accuracy')   # 검증 정확도
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(1, epochs + 1), train_loss_list, label='Training Loss')       # 훈련 손실
plt.plot(range(1, epochs + 1), val_loss_list, label='Validation Loss')       # 검증 손실
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# 테스트 데이터 평가
model.eval()  # 평가 모드
test_correct, test_total = 0, 0

with torch.no_grad():  # 추론 시 불필요한 gradient 계산 방지
    for x_batch, y_batch in test_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        outputs = model(x_batch).squeeze()             # 모델 예측(logit)
        preds = (outputs >= 0.5).float()               # 0.5 기준 이진 분류
        test_correct += (preds == y_batch).sum().item()  # 정확한 개수 누적
        test_total += y_batch.size(0)                    # 전체 개수 누적

test_acc = test_correct / test_total
print(f"테스트 정확도: {test_acc:.4f}")

## 5. 장단기 메모리 (Long Short-Term Memory, LSTM)

### 5.1. LSTM의 등장 배경
앞에서 본 것처럼, 기본 RNN은 시퀀스 데이터를 순서대로 처리하지만  
**입력이 길어질수록 앞쪽 정보가 뒤쪽까지 잘 전달되지 않는 한계**가 있습니다.

이처럼 과거의 정보가 나중 출력에 영향을 주어야 하는 상황을 **장기 의존성(Long-Term Dependency)** 문제라고 합니다.

기본 RNN은 학습 과정에서 기울기 소실 문제가 발생하기 쉬워,
중요한 정보를 오랫동안 유지하는 데 어려움이 있습니다.

LSTM(Long Short-Term Memory) 은 이러한 문제를 해결하기 위해 설계된 RNN의 변형으로,  
필요한 정보는 유지하고 불필요한 정보는 제거하도록 구조가 개선되었습니다.

 <img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FIe6mK%2FbtrrYR2JYzE%2FYBHMBjxLskjI7zETkmgrX0%2Fimg.png" width="600">

### 5.2. LSTM의 구조
 - LSTM의 핵심은 **셀 상태(cell state)** 와 **게이트(gates)** 입니다.  
셀 상태는 시퀀스의 모든 타임스텝을 거쳐 흐르는 정보의 주요 통로이며,  
불필요한 정보를 게이트가 걸러내고 중요한 정보만 남기는 역할을 합니다.  
이 때문에 필요한 정보는 시퀀스가 길어져도 잘 전달됩니다.

  LSTM은 다음과 같은 세 가지 게이트를 가지고 있습니다:

  - 입력 게이트(Input Gate): 새로운 정보가 셀 상태에 얼마나 추가될지를 결정합니다.
  - 망각 게이트(Forget Gate): 이전 셀 상태에서 어떤 정보를 버릴지를 결정합니다.
  - 출력 게이트(Output Gate): 현재 타임스텝에서 출력할 정보를 결정합니다.
  - <img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*J5W8FrASMi93Z81NlAui4w.png">

### 5.3. LSTM의 동작 과정

LSTM은 셀 상태(cell state)와 세 가지 게이트(망각 게이트, 입력 게이트, 출력 게이트)를 통해 정보를 처리하고 장기 의존성을 학습합니다.

#### 1. 망각 게이트 (Forget Gate)  
망각 게이트는 이전 타임스텝의 셀 상태에 어떤 정보를 남기고 어떤 정보를 버릴지 결정합니다.  

현재 입력 $x_t$와 이전 은닉 상태 $h_{t-1}$을 받아들여 시그모이드 함수로 변환해 값을 0과 1 사이로 조정하며,
0은 "완전히 버리기", 1은 "완전히 유지하기"를 의미합니다.  

$$
f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f)
$$

여기서 $f_t$는 각 정보의 유지 여부를 나타내는 벡터입니다.

#### 2. 입력 게이트 (Input Gate)  
입력 게이트는 현재 타임스텝에서 들어오는 정보를 셀 상태에 얼마나 추가할지를 결정합니다. 

이를 위해 시그모이드 함수와 하이퍼볼릭 탄젠트 함수를 사용합니다.

1. 시그모이드 함수로 중요한 정보 위치를 결정합니다.
   $$
   i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i)
   $$
   
2. 하이퍼볼릭 탄젠트 함수를 통해 새로운 후보 셀 상태를 만듭니다.
   $$
   \tilde{C}_t = \tanh(W_C \cdot [h_{t-1}, x_t] + b_C)
   $$

입력 게이트의 최종 결과는 $i_t $와 $\tilde{C}_t$의 곱으로, 새로운 정보가 얼마나 중요한지를 나타내는 벡터입니다.

#### 3. 셀 상태 업데이트  

이제 셀 상태를 업데이트합니다. 이전 셀 상태에 망각 게이트의 출력 f_t를 곱해 일부 정보는 잊고,  
입력 게이트의 결과를 더해 새로운 정보를 추가합니다.

$$
C_t = f_t * C_{t-1} + i_t * \tilde{C}_t
$$

#### 4. 출력 게이트 (Output Gate)  

출력 게이트는 다음 타임스텝에 전달할 은닉 상태 \( h_t \)를 결정합니다.

  1. 시그모이드 함수를 통해 출력할 정보의 중요도를 결정합니다.
   $$
   o_t = \sigma(W_o \cdot [h_{t-1}, x_t] + b_o)
   $$

  2. 현재 셀 상태 \( C_t \)에 하이퍼볼릭 탄젠트를 적용해 \( -1 \)과 \( 1 \) 사이로 제한하고, 여기에 \( o_t \)를 곱해 최종 은닉 상태를 만듭니다.
   $$
   h_t = o_t * \tanh(C_t)
   $$

이 은닉 상태 \( h_t \)는 출력으로 반환되며, 다음 타임스텝으로 전달됩니다.

<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*ahafyNt0Ph_J6Ed9_2hvdg.png">

<img src="https://miro.medium.com/v2/resize:fit:640/format:webp/0*O_TqfQ4537oM4MH0.gif">

### 5.4.LSTM의 장점
- 장기 의존성: 셀 상태가 정보의 주요 통로가 되어, 중요한 정보를 여러 타임스텝에 걸쳐 유지할 수 있어 장기 의존성 문제를 해결합니다.
- 기울기 소실 문제 해결: RNN에서 자주 발생하는 기울기 소실 문제를 방지하여, 더 깊고 긴 시퀀스에서도 안정적으로 학습이 가능합니다.

### 5.5.LSTM의 단점
- 복잡성: 일반 RNN에 비해 구조가 복잡하여 계산량이 많고, 훈련 시간과 메모리 요구 사항이 높습니다.
- 과적합 가능성: 파라미터 수가 많아 과적합의 위험이 있으며, 드롭아웃과 같은 과적합 방지 기법을 함께 적용하는 것이 중요합니다.

### 5.6.LSTM의 활용
- LSTM은 텍스트 생성, 번역, 시계열 예측, 감정 분석, 음성 인식 등 시퀀스 데이터를 다루는 다양한 분야에서 사용됩니다.

In [ ]:
# LSTM 모델 정의
class SimpleLSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden1, max_len):
        super(SimpleLSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm1 = nn.LSTM(embed_dim, hidden1, batch_first=True)
        self.dropout1 = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden1, 1)

    def forward(self, x):
        x = self.embedding(x)
        out1, _ = self.lstm1(x)
        out1 = self.dropout1(out1)
        last_output = out1[:, -1, :]   # 마지막 타임스텝
        out = self.fc(last_output)     # raw logit (sigmoid 없음!)
        return out


# 모델 생성
model = SimpleLSTMModel(max_features, embedding_dim, 256, max_len).to(device)
print(model)

In [ ]:
# 손실함수 & 옵티마이저
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 루프
train_acc_list, val_acc_list = [], []
train_loss_list, val_loss_list = [], []

for epoch in tqdm(range(epochs)):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for x_batch, y_batch in train_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(x_batch).squeeze()
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * x_batch.size(0)
        preds = (outputs >= 0.5).float()
        correct += (preds == y_batch).sum().item()
        total += y_batch.size(0)

    train_loss = running_loss / total
    train_acc = correct / total

    # 검증 단계
    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            outputs = model(x_batch).squeeze()
            loss = criterion(outputs, y_batch)
            val_loss += loss.item() * x_batch.size(0)
            preds = (outputs >= 0.5).float()
            val_correct += (preds == y_batch).sum().item()
            val_total += y_batch.size(0)

    val_loss /= val_total
    val_acc = val_correct / val_total

    train_acc_list.append(train_acc)
    val_acc_list.append(val_acc)
    train_loss_list.append(train_loss)
    val_loss_list.append(val_loss)

    print(f"Epoch [{epoch+1}/{epochs}] "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

In [ ]:
# 정확도/손실 시각화
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(range(1, epochs + 1), train_acc_list, label='Training Accuracy')
plt.plot(range(1, epochs + 1), val_acc_list, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(1, epochs + 1), train_loss_list, label='Training Loss')
plt.plot(range(1, epochs + 1), val_loss_list, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# 테스트 데이터 평가
model.eval()
test_correct, test_total = 0, 0
with torch.no_grad():
    for x_batch, y_batch in test_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        outputs = model(x_batch).squeeze()
        preds = (outputs >= 0.5).float()
        test_correct += (preds == y_batch).sum().item()
        test_total += y_batch.size(0)

test_acc = test_correct / test_total
print(f"테스트 정확도: {test_acc:.4f}")

### 체크포인트

#### 텍스트 데이터 전처리
- 텍스트 데이터는 특수문자, HTML 태그, 불필요한 공백 등 **노이즈**가 많아 정제가 필요하다.  
- 정규표현식(Regex)은 패턴 기반으로 텍스트를 효율적으로 처리할 수 있다.  
- 토큰화(Tokenization)는 긴 문장을 **모델이 처리 가능한 단위**로 쪼개는 과정이다.  
- 현대 NLP에서는 불용어 제거를 거의 하지 않으며, Subword 토큰화(BPE, SentencePiece)를 통해 **OOV 문제**를 완화한다.  

#### 임베딩 (Embedding)
- 원-핫 인코딩은 희소 벡터라서 단어 간 의미를 반영하지 못한다.  
- 임베딩은 밀집 벡터로 의미적 유사성을 표현하며, **단어 간 관계를 수치화**한다.  
- BoW/TF-IDF/N-gram은 통계 기반 접근으로 단순하지만, 의미적 정보는 부족하다.  
- Word2Vec은 CBOW/Skip-Gram을 통해 의미적 유사성을 반영한다.  
- FastText는 Subword 단위를 사용하여 희귀 단어와 신조어에도 강하다.  

#### 순환 신경망 (RNN & LSTM)
- RNN은 시퀀스 데이터를 순차적으로 처리하며, 은닉 상태(hidden state)로 과거 정보를 기억한다.  
- RNN은 **장기 의존성 문제(vanishing gradient)** 로 긴 문맥 처리가 어렵다.  
- LSTM은 게이트 구조(input, forget, output gate)를 추가하여 **장기 의존성 문제를 해결**하고 긴 문맥도 학습할 수 있다.  
- RNN/LSTM은 텍스트 분류, 품사 태깅, 감성 분석 등 시퀀스 학습에 활용된다.  

#### 종합
- NLP 파이프라인은 **데이터 정제 → 토큰화 → 임베딩 → RNN/LSTM 모델링** 흐름으로 구성된다.  
- 전통적 통계 기반 임베딩에서 분산 표현, 그리고 RNN/LSTM으로 발전하면서 **언어의 문맥과 의미를 더 깊이 반영**할 수 있게 되었다.  
- 이후 Transformer 구조는 RNN/LSTM의 한계를 극복하는 방향으로 발전하였다.

### 도전 과제: LSTM으로 주식 가격 예측해보기

이번에는 지금까지 배운 **시계열 + RNN/LSTM** 개념을 실제 금융 데이터에 적용해 보는 도전 과제입니다.  
주식 가격은 시간에 따라 변화하는 **시계열 데이터**이기 때문에, LSTM으로 이전 며칠 간의 가격 → 다음 날 가격(또는 상승/하락)을 예측해 볼 수 있습니다.

#### 1. FinanceDataReader 설치

먼저 한국/미국 등 다양한 시장의 주가 데이터를 쉽게 가져올 수 있는 `FinanceDataReader` 라이브러리를 설치해 주세요.

```bash
pip install finance-datareader
```

#### 2. 주가 데이터 조회 예시

아래 예시는 삼성전자(종목코드 `005930`)의 일별 주가를 불러오는 코드입니다.  
원하는 다른 종목(예: KOSPI 지수 `KS11`, 애플 `AAPL` 등)으로 바꿔서 실험해도 좋습니다.

In [ ]:
import FinanceDataReader as fdr

# 예시 1: 삼성전자(005930) 2015년 이후 일별 주가
df = fdr.DataReader('005930', '2015-01-01')  # 종료일 생략 시 오늘까지
print(df.head())
print(df.columns)  # ['Open', 'High', 'Low', 'Close', 'Volume', ...] 등 확인

# 예시 2: KOSPI 지수 (KS11)
kospi = fdr.DataReader('KS11', '2015-01-01')
print(kospi.head())

#### 3. 미션: LSTM으로 주가 예측 모델 만들기

아래 과정을 스스로 설계해서, **간단한 LSTM 기반 주가 예측 모델**을 만들어 보세요.

- (1) 사용할 열 선택: 예를 들어 종가(`Close`)만 사용하거나, `Open/High/Low/Close/Volume` 일부를 선택  
- (2) 정규화/스케일링: `MinMaxScaler` 등으로 0~1 범위로 스케일링  
- (3) 시퀀스 만들기:  
  - 최근 `N`일(예: 20일)의 데이터를 입력으로 하고  
  - 그 다음 날(또는 다음 며칠)의 종가/상승·하락 여부를 라벨로 만드는 방식  
- (4) LSTM 모델 정의 (PyTorch, Keras 등 아무 프레임워크나 사용 가능)  
- (5) 학습 및 평가:  
  - MSE/RMSE 등 회귀 지표  
  - 또는 “상승/하락”을 분류 문제로 바꾸어 정확도/정밀도 등을 볼 수도 있음  
- (6) 예측 결과를 시각화해 실제 가격 곡선과 비교해 보기  

> **힌트**: IMDB 감성 분류 실습에서 사용한 **시퀀스 패딩, DataLoader 구성, LSTM 모델 정의/학습 루프** 구조를 재활용하면 구현이 한결 쉬워집니다.

#### 4. (선택) 뉴스까지 활용해 보고 싶은 분들을 위한 아이디어

도전을 더 해보고 싶은 분들은, **뉴스 텍스트 + 주가를 함께 사용하는 예측**도 시도해 볼 수 있습니다.

- 금융/증권 뉴스 헤드라인 또는 기사 본문을 수집하고, 해당 날짜의 주가와 매칭  
- 뉴스 텍스트를  
  - 전통 방식: TF-IDF + 간단 모델  
  - 또는 임베딩 + LSTM/Transformer 로 인코딩한 뒤,  
- 가격 시계열 특징과 함께 합쳐서 “다음 날 상승/하락”을 예측하는 모델 설계  

※ 실제로는 매우 어려운 문제이니, **성능보다는 파이프라인 구성 경험**에 초점을 두고 도전해 보세요.
검색을 통해 다양한 예시를 참고해도 좋습니다.